# SunCloud Safety Layer

SunCloud Safety Layer

Prepared by: High Street Consulting Group for AZ MAG SunCloud

Contact: Yousef Dana, PE - dana@highstreetconsulting.com

Dates: 2022 - 2023

Purpose: This script creates junction and segment safety layers for the Sun Corridor area, initially for the SunCloud project. 
    You can find direct links to the published 2021 layers below:
    
    Junctions: https://azgeo.maps.arcgis.com/home/item.html?id=b7ab2303e3bd4e58a1e0a8964b07a951
    Segments: https://azgeo.maps.arcgis.com/home/item.html?id=fdb09b5629514b6fadea59c86b764045
        
Notes:
    An internet connection is required to access external web data.
    If running within ArcGIS Pro, depending on your PC's processing power it may give an error that a lock cannot aquire, please just re-run the chunk.

# Variables

In [ ]:
# Please adjust the values within these parameters as needed:

# Define start and end years (Typically there is a 5-year analysis)
start_year = 2018
end_year = 2022



## Setup

In [ ]:
import arcpy
import os
import pandas as pd
import math
import re

arcpy.env.workspace = None

# Setup arcpy environment
current_working_directory = os.getcwd()  # Get the current working directory

if arcpy.env.workspace:  # Check if workspace is not None
    base_path = '\\'.join(arcpy.env.workspace.split('\\')[:-1])
else:
    print("Workspace was not set. Setting it now.")
    base_path = current_working_directory  # Initialize workspace to the current working directory
    arcpy.env.workspace = base_path

db = os.path.join(base_path, "safety_layers.gdb")
arcpy.env.workspace = db
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# Print the current base path directory
print(f"Base path and workspace directory: {base_path}")  # Debug

# Import the GIS class in gis module
from arcgis.gis import GIS
gis = GIS("pro")
print("Logged in as " + str(gis.properties.user.username) + " to " + gis.properties.portalName + " through the portal: " + gis.properties.urlKey)

# Check if logged into the correct portal
expected_portal = "azgeo"
if gis.properties.urlKey != expected_portal:
    print(f"WARNING: You are logged into the wrong portal. Please switch to the AZGeo Data Hub at https://azgeo.maps.arcgis.com/")
else:
    print("Successfully initiated.")


## Code

### Load Data

In [ ]:
## Combine all crashes

# Read and combine incident-crash data for the specified range of years
crashes = pd.DataFrame()

for year in range(start_year, end_year + 1):
    year_file_path = os.path.join(base_path, "data", "crash", str(year), f"Incident_{year}.csv")
    year_data = pd.read_csv(year_file_path)
    crashes = pd.concat([crashes, year_data])

# Write to csv
crashes.to_csv(os.path.join(base_path, "data", "crash", "crashes.csv"), index=False)

# Calculate the number of years including start year
years_combined = end_year - start_year + 1

# Print the message
print(f"Successfully combined {years_combined} years of crash data")


In [ ]:
# Load crash data into spatial points layer in the database using XY to Point
prj_4326 = arcpy.SpatialReference(4326)

in_table = os.path.join(base_path, "data", "crash", "crashes.csv")
out_table = os.path.join("safety_layers.gdb", "crashes")

arcpy.management.XYTableToPoint(in_table, 
                                out_table, 
                                "Longitude", 
                                "Latitude", 
                                None, 
                                prj_4326)

In [ ]:
## Combine non-motorized serious injury crashes

# Read and combine non-motorized serious injury person-crash data for the specified range of years
person = pd.DataFrame()

for year in range(start_year, end_year + 1):
    year_file_path = os.path.join(base_path, "data", "crash", str(year), f"Person_{year}.csv")
    year_data = pd.read_csv(year_file_path)
    person = pd.concat([person, year_data])

# Write person data to csv
person_output_path = os.path.join(base_path, "data", "crash", "person.csv")
person.to_csv(person_output_path, index=False)

# Filtering based on 'PersonType' and 'InjuryStatus'
ped_injury_person = person[(person['PersonType'].isin([2, 3])) & (person['InjuryStatus'] == 4)]

# Creating a new DataFrame 'ped_injury_crashes' with unique 'IncidentID' and its count
ped_injury_crashes = pd.DataFrame(ped_injury_person['IncidentID'].value_counts()).reset_index()

# Renaming the columns appropriately
ped_injury_crashes.columns = ['IncidentID', 'non_motorized_serious_injuries']

# Write ped_injury_crashes data to csv
ped_injury_crashes_output_path = os.path.join(base_path, "data", "crash", "ped_injury_crashes.csv")
ped_injury_crashes.to_csv(ped_injury_crashes_output_path, index=False)

In [ ]:
# Import the CSV file as a table
ped_injury_crashes = os.path.join(base_path, "data", "crash", "ped_injury_crashes.csv")

arcpy.TableToTable_conversion(ped_injury_crashes, 
                              arcpy.env.workspace, 
                              "ped_injury_crashes")

# Join crashes and non-motorized serious injuries
arcpy.JoinField_management("crashes", "IncidentID", 
                           "ped_injury_crashes", "IncidentID", 
                           "non_motorized_serious_injuries")


In [ ]:
# This takes about 1 hour to read/load each layer into the database. ATIS is the longest.

# Load non-crash data into a feature layers in the database
from arcgis.features import FeatureLayer

# County Boundary
url_fl =  "https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/counties/FeatureServer/0"
fl = FeatureLayer(url_fl)
featureset = fl.query()
featureset.save(db, "counties")

# AADT 2019
url_fl =  "https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/AADT_2020_gdb/FeatureServer/0"
fl = FeatureLayer(url_fl)
featureset = fl.query()
featureset.save(db, "aadt")

# Urban Areas
url_fl =  "https://services.arcgis.com/P3ePLMYs2RVChkJx/ArcGIS/rest/services/USA_Urban_Areas/FeatureServer/3"
fl = FeatureLayer(url_fl)
featureset = fl.query()
featureset.save(db, "urban_areas")

# Median layer for Divided/Undivided attributes
url_fl =  "https://services1.arcgis.com/XAiBIVuto7zeZj1B/arcgis/rest/services/ATIS_prod_gdb/FeatureServer/33"
fl = FeatureLayer(url_fl)
featureset = fl.query()
featureset.save(db, "median")

# ATIS 2021
url_fl =  "https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/AZ_All_Roads_Network_2021/FeatureServer/0"
fl = FeatureLayer(url_fl)
featureset = fl.query()
featureset.save(db, "atis")

# HPMS Through Lanes
url_fl =  "https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/Arizona_Highway_Performance_Monitoring_System_2020_Data_/FeatureServer/41"
fl = FeatureLayer(url_fl)
featureset = fl.query()
featureset.save(db, "through_lanes")  


In [ ]:
# Load segmented routes and major junction layers created by High Street

# https://azgeo.maps.arcgis.com/home/item.html?id=9a948ae2bcb84925bfdc64e90128f715
# Segments
url_fl =  "https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/Sun_Cloud_Segmented_Routes/FeatureServer/0"
fl = FeatureLayer(url_fl)
featureset = fl.query()
featureset.save(db, "segments")

# https://azgeo.maps.arcgis.com/home/item.html?id=4b093a019ea3467e874a382be442255c
# Junctions
url_fl =  "https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/Sun_Cloud_Major_Junctions_DEV/FeatureServer/0"
fl = FeatureLayer(url_fl)
featureset = fl.query()
featureset.save(db, "junctions")

In [ ]:
# Join HPMS to segments to get Number of Lanes
arcpy.analysis.SpatialJoin("segments", "through_lanes", r"safety_layers.gdb\segments_0", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'route_id "Route ID" true true false 32 Text 0 0,First,#,segments,route_id,0,32;functional_classification_code "Functional Classification Code" true true false 2 Short 0 0,First,#,segments,functional_classification_code,-1,-1;functional_classification "Functional Classification" true true false 255 Text 0 0,First,#,segments,functional_classification,0,255;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,segments,Shape_Length,-1,-1;NumberOfLanes "NumberOfLanes" true true false 8 Double 0 0,First,#,through_lanes,NumberOfLanes,-1,-1', "LARGEST_OVERLAP",  "150 Feet", '')

### Clip to Boundary

In [ ]:
# Filter counties to just suncloud area:
input_layer = "counties"
output_layer = "counties_suncloud"
query = "NAME = 'MARICOPA' OR NAME = 'PIMA' OR NAME = 'PINAL' OR NAME = 'COCHISE' OR NAME = 'SANTA CRUZ'"

# Select features based on the SQL query and copy
arcpy.SelectLayerByAttribute_management(input_layer, "NEW_SELECTION", query)
arcpy.CopyFeatures_management(input_layer, output_layer)

# Clear selection if necessary
arcpy.SelectLayerByAttribute_management(input_layer, "CLEAR_SELECTION")

In [ ]:
# Clip Crashes to the Boundary
# Not needed for other data as they have been clipped prior to this script

points = r"safety_layers.gdb\crashes"
polygon = r"safety_layers.gdb\counties_suncloud"

# Clip crashes to the selected (filtered) counties
arcpy.analysis.Clip(points, polygon, 
                    r"safety_layers.gdb\clipped_crashes", None)


### Safety Calculations

#### Crash Frequency  

In [ ]:
## Join crashes to Junctions and Segments

# Query to only crashes that occured with 150 feet of a junction.
field_mappings = arcpy.FieldMappings()
field_mappings.addTable("clipped_crashes")
field_mappings.addTable("junctions")

arcpy.analysis.SpatialJoin("clipped_crashes", "junctions",
                           r"safety_layers.gdb\crashes_joined",
                           "JOIN_ONE_TO_ONE",
                           "KEEP_ALL",
                           field_mappings,
                           "WITHIN_A_DISTANCE",
                           "150 FeetInt",
                           '')


arcpy.management.MakeQueryTable("crashes_joined", "junction_crashes", 
                                "USE_KEY_FIELDS", 
                                None, 
                                None, 
                                "crashes_joined.MasterJunc IS NOT NULL")

arcpy.management.MakeQueryTable("crashes_joined", "segment_crashes", 
                                "USE_KEY_FIELDS", 
                                None, 
                                None, 
                                "crashes_joined.MasterJunc IS NULL")


In [ ]:
# Crashes to Junctions
# For crashes WITH junctions within 150 feet. 
# Count the total crashes, sum the number of fatalities, injuries, and non-motorized fatalities+injuries.
arcpy.analysis.SpatialJoin("junctions", "junction_crashes", 
                           r"safety_layers.gdb\junctions_1", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", 
                           'FID_Junctions_SunCloud_Select "FID_Junctions_SunCloud_Select" true true false 4 Long 0 0,First,#,junctions,FID_Junctions_SunCloud_Select,-1,-1;FromDate "FromDate" true true false 8 Date 0 0,First,#,junctions,FromDate,-1,-1;ToDate "ToDate" true true false 8 Date 0 0,First,#,junctions,ToDate,-1,-1;RouteID "RouteID" true true false 32 Text 0 0,First,#,junctions,RouteID,0,32;Measure "Measure" true true false 8 Double 0 0,First,#,junctions,Measure,-1,-1;MasterJunc "MasterJunc" true true false 38 Text 0 0,First,#,junctions,MasterJunc,0,38;NumberOfLe "NumberOfLe" true true false 4 Long 0 0,First,#,junctions,NumberOfLe,-1,-1;JunctionGe "JunctionGe" true true false 4 Long 0 0,First,#,junctions,JunctionGe,-1,-1;JunctionsS "JunctionsS" true true false 2 Text 0 0,First,#,junctions,JunctionsS,0,2;TrafficCon "TrafficCon" true true false 4 Long 0 0,First,#,junctions,TrafficCon,-1,-1;TurnCode "TurnCode" true true false 10 Text 0 0,First,#,junctions,TurnCode,0,10;Detection "Detection" true true false 4 Long 0 0,First,#,junctions,Detection,-1,-1;Signalizat "Signalizat" true true false 4 Long 0 0,First,#,junctions,Signalizat,-1,-1;RailCrossi "RailCrossi" true true false 7 Text 0 0,First,#,junctions,RailCrossi,0,7;SchoolZone "SchoolZone" true true false 4 Long 0 0,First,#,junctions,SchoolZone,-1,-1;BicycleFac "BicycleFac" true true false 4 Long 0 0,First,#,junctions,BicycleFac,-1,-1;Crosswalk "Crosswalk" true true false 4 Long 0 0,First,#,junctions,Crosswalk,-1,-1;PctGreenTi "PctGreenTi" true true false 4 Long 0 0,First,#,junctions,PctGreenTi,-1,-1;PctGreen_1 "PctGreen_1" true true false 4 Long 0 0,First,#,junctions,PctGreen_1,-1,-1;SourceID "SourceID" true true false 4 Long 0 0,First,#,junctions,SourceID,-1,-1;JunctionMa "JunctionMa" true true false 4 Long 0 0,First,#,junctions,JunctionMa,-1,-1;JunctionTy "JunctionTy" true true false 4 Long 0 0,First,#,junctions,JunctionTy,-1,-1;OffsetInte "OffsetInte" true true false 1 Text 0 0,First,#,junctions,OffsetInte,0,1;OffsetIn_1 "OffsetIn_1" true true false 8 Double 0 0,First,#,junctions,OffsetIn_1,-1,-1;InServiceD "InServiceD" true true false 8 Date 0 0,First,#,junctions,InServiceD,-1,-1;FID_sun_cloud_routes "FID_sun_cloud_routes" true true false 4 Long 0 0,First,#,junctions,FID_sun_cloud_routes,-1,-1;route_id "route_id" true true false 32 Text 0 0,First,#,junctions,route_id,0,32;from_measu "from_measu" true true false 8 Double 0 0,First,#,junctions,from_measu,-1,-1;to_measure "to_measure" true true false 8 Double 0 0,First,#,junctions,to_measure,-1,-1;functional "functional" true true false 4 Long 0 0,First,#,junctions,functional,-1,-1;facility_t "facility_t" true true false 4 Long 0 0,First,#,junctions,facility_t,-1,-1;crashes "crashes" true true false 255 Text 0 0,Count,#,junction_crashes,TotalUnits,-1,-1;injury_crashes "injury_crashes" true true false 255 Double 0 0,Sum,#,junction_crashes,TotalInjuries,-1,-1;fatal_crashes "fatal_crashes" true true false 255 Double 0 0,Sum,#,junction_crashes,TotalFatalities,-1,-1;fatal_injury_crashes "fatal_injury_crashes" true true false 255 Double 0 0,Sum,#,junction_crashes,TotalInjuries,-1,-1,junction_crashes,TotalFatalities,-1,-1;non_motorized_fatal_injury "non_motorized_fatal_injury" true true false 255 Double 0 0,Sum,#,junction_crashes,non_motorized_serious_injuries,-1,-1,junction_crashes,TotalNonMotoristsFatalities,-1,-1', 
                           "WITHIN_A_DISTANCE", "150 Feet", '')

# Crashes to Segments

# Make a copy
arcpy.conversion.FeatureClassToFeatureClass("segment_crashes", r"safety_layers.gdb", "segment_crashes_1", '', 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,segment_crashes,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,segment_crashes,TARGET_FID,-1,-1;IncidentID "IncidentID" true true false 4 Long 0 0,First,#,segment_crashes,IncidentID,-1,-1;Microfilm "Microfilm" true true false 8000 Text 0 0,First,#,segment_crashes,Microfilm,0,8000;ADOTReceivedDate "ADOTReceivedDate" true true false 8000 Text 0 0,First,#,segment_crashes,ADOTReceivedDate,0,8000;Status "Status" true true false 4 Long 0 0,First,#,segment_crashes,Status,-1,-1;DataConv2008Flag "DataConv2008Flag" true true false 8000 Text 0 0,First,#,segment_crashes,DataConv2008Flag,0,8000;IncidentDate "IncidentDate" true true false 8000 Text 0 0,First,#,segment_crashes,IncidentDate,0,8000;IncidentDateTime "IncidentDateTime" true true false 8000 Text 0 0,First,#,segment_crashes,IncidentDateTime,0,8000;IncidentYear "IncidentYear" true true false 4 Long 0 0,First,#,segment_crashes,IncidentYear,-1,-1;IncidentMonth "IncidentMonth" true true false 4 Long 0 0,First,#,segment_crashes,IncidentMonth,-1,-1;IncidentDayOfWeek "IncidentDayOfWeek" true true false 4 Long 0 0,First,#,segment_crashes,IncidentDayOfWeek,-1,-1;IncidentHour "IncidentHour" true true false 4 Long 0 0,First,#,segment_crashes,IncidentHour,-1,-1;OfficerNcic "OfficerNcic" true true false 4 Long 0 0,First,#,segment_crashes,OfficerNcic,-1,-1;OfficerId "OfficerId" true true false 8000 Text 0 0,First,#,segment_crashes,OfficerId,0,8000;FileNumber "FileNumber" true true false 8000 Text 0 0,First,#,segment_crashes,FileNumber,0,8000;ExtendedNcic "ExtendedNcic" true true false 4 Long 0 0,First,#,segment_crashes,ExtendedNcic,-1,-1;DamageOverLimitFlag "DamageOverLimitFlag" true true false 8000 Text 0 0,First,#,segment_crashes,DamageOverLimitFlag,0,8000;PhotosTakenFlag "PhotosTakenFlag" true true false 8000 Text 0 0,First,#,segment_crashes,PhotosTakenFlag,0,8000;PhotographerName "PhotographerName" true true false 8000 Text 0 0,First,#,segment_crashes,PhotographerName,0,8000;PhotographerID "PhotographerID" true true false 8000 Text 0 0,First,#,segment_crashes,PhotographerID,0,8000;PhotographerAgency "PhotographerAgency" true true false 8000 Text 0 0,First,#,segment_crashes,PhotographerAgency,0,8000;InvestigatedAtSceneFlag "InvestigatedAtSceneFlag" true true false 8000 Text 0 0,First,#,segment_crashes,InvestigatedAtSceneFlag,0,8000;DateTimeInvestigated "DateTimeInvestigated" true true false 8000 Text 0 0,First,#,segment_crashes,DateTimeInvestigated,0,8000;CollisionManner "CollisionManner" true true false 4 Long 0 0,First,#,segment_crashes,CollisionManner,-1,-1;LightCondition "LightCondition" true true false 4 Long 0 0,First,#,segment_crashes,LightCondition,-1,-1;FirstHarmfulEvent "FirstHarmfulEvent" true true false 4 Long 0 0,First,#,segment_crashes,FirstHarmfulEvent,-1,-1;FirstHarmfulLocation "FirstHarmfulLocation" true true false 8000 Text 0 0,First,#,segment_crashes,FirstHarmfulLocation,0,8000;TotalUnits "TotalUnits" true true false 4 Long 0 0,First,#,segment_crashes,TotalUnits,-1,-1;TotalMotorists "TotalMotorists" true true false 4 Long 0 0,First,#,segment_crashes,TotalMotorists,-1,-1;TotalNonMotorists "TotalNonMotorists" true true false 4 Long 0 0,First,#,segment_crashes,TotalNonMotorists,-1,-1;TotalInjuries "TotalInjuries" true true false 4 Long 0 0,First,#,segment_crashes,TotalInjuries,-1,-1;TotalFatalities "TotalFatalities" true true false 4 Long 0 0,First,#,segment_crashes,TotalFatalities,-1,-1;TotalMotorists1 "TotalMotorists1" true true false 4 Long 0 0,First,#,segment_crashes,TotalMotorists1,-1,-1;TotalNonMotorists1 "TotalNonMotorists1" true true false 4 Long 0 0,First,#,segment_crashes,TotalNonMotorists1,-1,-1;TotalMotoristsFatalities "TotalMotoristsFatalities" true true false 4 Long 0 0,First,#,segment_crashes,TotalMotoristsFatalities,-1,-1;TotalNonMotoristsFatalities "TotalNonMotoristsFatalities" true true false 4 Long 0 0,First,#,segment_crashes,TotalNonMotoristsFatalities,-1,-1;InjurySeverity "InjurySeverity" true true false 4 Long 0 0,First,#,segment_crashes,InjurySeverity,-1,-1;MedicalTransportFlag "MedicalTransportFlag" true true false 8000 Text 0 0,First,#,segment_crashes,MedicalTransportFlag,0,8000;FatalAccidentFlag "FatalAccidentFlag" true true false 4 Long 0 0,First,#,segment_crashes,FatalAccidentFlag,-1,-1;TowAwayFlag "TowAwayFlag" true true false 4 Long 0 0,First,#,segment_crashes,TowAwayFlag,-1,-1;NscReportable "NscReportable" true true false 8000 Text 0 0,First,#,segment_crashes,NscReportable,0,8000;SchoolBusRelatedFlag "SchoolBusRelatedFlag" true true false 4 Long 0 0,First,#,segment_crashes,SchoolBusRelatedFlag,-1,-1;WorkZoneRelatedFlag "WorkZoneRelatedFlag" true true false 4 Long 0 0,First,#,segment_crashes,WorkZoneRelatedFlag,-1,-1;WorkersPresentFlag "WorkersPresentFlag" true true false 4 Long 0 0,First,#,segment_crashes,WorkersPresentFlag,-1,-1;AlcoholInvolvementFlag "AlcoholInvolvementFlag" true true false 4 Long 0 0,First,#,segment_crashes,AlcoholInvolvementFlag,-1,-1;DrugInvolvementFlag "DrugInvolvementFlag" true true false 4 Long 0 0,First,#,segment_crashes,DrugInvolvementFlag,-1,-1;HazardousFlag "HazardousFlag" true true false 8000 Text 0 0,First,#,segment_crashes,HazardousFlag,0,8000;HitAndRunFlag "HitAndRunFlag" true true false 4 Long 0 0,First,#,segment_crashes,HitAndRunFlag,-1,-1;RouteName "RouteName" true true false 8000 Text 0 0,First,#,segment_crashes,RouteName,0,8000;CrossingFeatureName "CrossingFeatureName" true true false 8000 Text 0 0,First,#,segment_crashes,CrossingFeatureName,0,8000;Offset "Offset" true true false 8 Double 0 0,First,#,segment_crashes,Offset,-1,-1;OffsetMeasuredFlag "OffsetMeasuredFlag" true true false 4 Long 0 0,First,#,segment_crashes,OffsetMeasuredFlag,-1,-1;OffsetUnit "OffsetUnit" true true false 4 Long 0 0,First,#,segment_crashes,OffsetUnit,-1,-1;MPNum "MPNum" true true false 4 Long 0 0,First,#,segment_crashes,MPNum,-1,-1;MPOffset "MPOffset" true true false 8 Double 0 0,First,#,segment_crashes,MPOffset,-1,-1;CityId "CityId" true true false 4 Long 0 0,First,#,segment_crashes,CityId,-1,-1;CountyId "CountyId" true true false 4 Long 0 0,First,#,segment_crashes,CountyId,-1,-1;StateId "StateId" true true false 4 Long 0 0,First,#,segment_crashes,StateId,-1,-1;StateCode "StateCode" true true false 8000 Text 0 0,First,#,segment_crashes,StateCode,0,8000;CountryCode "CountryCode" true true false 8000 Text 0 0,First,#,segment_crashes,CountryCode,0,8000;Latitude "Latitude" true true false 8 Double 0 0,First,#,segment_crashes,Latitude,-1,-1;Longitude "Longitude" true true false 8 Double 0 0,First,#,segment_crashes,Longitude,-1,-1;ValidLocationFlag "ValidLocationFlag" true true false 4 Long 0 0,First,#,segment_crashes,ValidLocationFlag,-1,-1;LocationToolVersion "LocationToolVersion" true true false 8000 Text 0 0,First,#,segment_crashes,LocationToolVersion,0,8000;HESIntersectionFlag "HESIntersectionFlag" true true false 8000 Text 0 0,First,#,segment_crashes,HESIntersectionFlag,0,8000;IntersectionFlag "IntersectionFlag" true true false 4 Long 0 0,First,#,segment_crashes,IntersectionFlag,-1,-1;IntersectionRouteID "IntersectionRouteID" true true false 8000 Text 0 0,First,#,segment_crashes,IntersectionRouteID,0,8000;IntersectionATISCode "IntersectionATISCode" true true false 8000 Text 0 0,First,#,segment_crashes,IntersectionATISCode,0,8000;TrafficWayType "TrafficWayType" true true false 4 Long 0 0,First,#,segment_crashes,TrafficWayType,-1,-1;IntersectionType "IntersectionType" true true false 4 Long 0 0,First,#,segment_crashes,IntersectionType,-1,-1;JunctionRelation "JunctionRelation" true true false 4 Long 0 0,First,#,segment_crashes,JunctionRelation,-1,-1;Weather "Weather" true true false 4 Long 0 0,First,#,segment_crashes,Weather,-1,-1;PropertyDamageOwnerCode "PropertyDamageOwnerCode" true true false 8 Double 0 0,First,#,segment_crashes,PropertyDamageOwnerCode,-1,-1;PropertyDamageDescCode "PropertyDamageDescCode" true true false 8 Double 0 0,First,#,segment_crashes,PropertyDamageDescCode,-1,-1;OffsetDirection "OffsetDirection" true true false 4 Long 0 0,First,#,segment_crashes,OffsetDirection,-1,-1;SecondaryCrashFlag "SecondaryCrashFlag" true true false 4 Long 0 0,First,#,segment_crashes,SecondaryCrashFlag,-1,-1;GeocodeOnRoad "GeocodeOnRoad" true true false 8000 Text 0 0,First,#,segment_crashes,GeocodeOnRoad,0,8000;GeocodeCrossingFeatureName "GeocodeCrossingFeatureName" true true false 8000 Text 0 0,First,#,segment_crashes,GeocodeCrossingFeatureName,0,8000;GeocodeOffsetInMiles "GeocodeOffsetInMiles" true true false 8 Double 0 0,First,#,segment_crashes,GeocodeOffsetInMiles,-1,-1;X "X" true true false 8 Double 0 0,First,#,segment_crashes,X,-1,-1;Y "Y" true true false 8 Double 0 0,First,#,segment_crashes,Y,-1,-1;GeocodeOnRoadRouteId "GeocodeOnRoadRouteId" true true false 8000 Text 0 0,First,#,segment_crashes,GeocodeOnRoadRouteId,0,8000;CrossingFeature "CrossingFeature" true true false 8000 Text 0 0,First,#,segment_crashes,CrossingFeature,0,8000;non_motorized_serious_injuries "non_motorized_serious_injuries" true true false 4 Long 0 0,First,#,segment_crashes,non_motorized_serious_injuries,-1,-1;FID_Junctions_SunCloud_Select "FID_Junctions_SunCloud_Select" true true false 4 Long 0 0,First,#,segment_crashes,FID_Junctions_SunCloud_Select,-1,-1;FromDate "FromDate" true true false 8 Date 0 0,First,#,segment_crashes,FromDate,-1,-1;ToDate "ToDate" true true false 8 Date 0 0,First,#,segment_crashes,ToDate,-1,-1;RouteID "RouteID" true true false 32 Text 0 0,First,#,segment_crashes,RouteID,0,32;Measure "Measure" true true false 8 Double 0 0,First,#,segment_crashes,Measure,-1,-1;MasterJunc "MasterJunc" true true false 38 Text 0 0,First,#,segment_crashes,MasterJunc,0,38;NumberOfLe "NumberOfLe" true true false 4 Long 0 0,First,#,segment_crashes,NumberOfLe,-1,-1;JunctionGe "JunctionGe" true true false 4 Long 0 0,First,#,segment_crashes,JunctionGe,-1,-1;JunctionsS "JunctionsS" true true false 2 Text 0 0,First,#,segment_crashes,JunctionsS,0,2;TrafficCon "TrafficCon" true true false 4 Long 0 0,First,#,segment_crashes,TrafficCon,-1,-1;TurnCode "TurnCode" true true false 10 Text 0 0,First,#,segment_crashes,TurnCode,0,10;Detection "Detection" true true false 4 Long 0 0,First,#,segment_crashes,Detection,-1,-1;Signalizat "Signalizat" true true false 4 Long 0 0,First,#,segment_crashes,Signalizat,-1,-1;RailCrossi "RailCrossi" true true false 7 Text 0 0,First,#,segment_crashes,RailCrossi,0,7;SchoolZone "SchoolZone" true true false 4 Long 0 0,First,#,segment_crashes,SchoolZone,-1,-1;BicycleFac "BicycleFac" true true false 4 Long 0 0,First,#,segment_crashes,BicycleFac,-1,-1;Crosswalk "Crosswalk" true true false 4 Long 0 0,First,#,segment_crashes,Crosswalk,-1,-1;PctGreenTi "PctGreenTi" true true false 4 Long 0 0,First,#,segment_crashes,PctGreenTi,-1,-1;PctGreen_1 "PctGreen_1" true true false 4 Long 0 0,First,#,segment_crashes,PctGreen_1,-1,-1;SourceID "SourceID" true true false 4 Long 0 0,First,#,segment_crashes,SourceID,-1,-1;JunctionMa "JunctionMa" true true false 4 Long 0 0,First,#,segment_crashes,JunctionMa,-1,-1;JunctionTy "JunctionTy" true true false 4 Long 0 0,First,#,segment_crashes,JunctionTy,-1,-1;OffsetInte "OffsetInte" true true false 1 Text 0 0,First,#,segment_crashes,OffsetInte,0,1;OffsetIn_1 "OffsetIn_1" true true false 8 Double 0 0,First,#,segment_crashes,OffsetIn_1,-1,-1;InServiceD "InServiceD" true true false 8 Date 0 0,First,#,segment_crashes,InServiceD,-1,-1;FID_sun_cloud_routes "FID_sun_cloud_routes" true true false 4 Long 0 0,First,#,segment_crashes,FID_sun_cloud_routes,-1,-1;route_id "route_id" true true false 32 Text 0 0,First,#,segment_crashes,route_id,0,32;from_measu "from_measu" true true false 8 Double 0 0,First,#,segment_crashes,from_measu,-1,-1;to_measure "to_measure" true true false 8 Double 0 0,First,#,segment_crashes,to_measure,-1,-1;functional "functional" true true false 4 Long 0 0,First,#,segment_crashes,functional,-1,-1;facility_t "facility_t" true true false 4 Long 0 0,First,#,segment_crashes,facility_t,-1,-1;Shape__Len "Shape__Len" true true false 8 Double 0 0,First,#,segment_crashes,Shape__Len,-1,-1', '')


In [ ]:
# Snap crashes to segments within 150 feet by edge
arcpy.edit.Snap("segment_crashes_1", "segments EDGE '150 Feet'")

# For crashes WITHOUT junctions within 150 feet. 
# Count the total crashes, sum the number of fatalities, injuries, and non-motorized fatalities+injuries.
arcpy.analysis.SpatialJoin("segments_0", "segment_crashes_1", 
                           r"safety_layers.gdb\segments_1", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", 
                           'route_id "Route ID" true true false 32 Text 0 0,First,#,segments,route_id,0,32;functional_classification_code "Functional Classification Code" true true false 2 Short 0 0,First,#,segments,functional_classification_code,-1,-1;functional_classification "Functional Classification" true true false 255 Text 0 0,First,#,segments,functional_classification,0,255;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,segments,Shape_Length,-1,-1;crashes "crashes" true true false 255 Double 0 0,Count,#,segment_crashes,TotalUnits,-1,-1;injury_crashes "injury_crashes" true true false 255 Double 0 0,Sum,#,segment_crashes,TotalInjuries,-1,-1;fatal_crashes "fatal_crashes" true true false 255 Double 0 0,Sum,#,segment_crashes,TotalFatalities,-1,-1;fatal_injury_crashes "fatal_injury_crashes" true true false 255 Double 0 0,Sum,#,segment_crashes,TotalFatalities,-1,-1,segment_crashes,TotalInjuries,-1,-1;non_motorized_fatal_injury_crashes "non_motorized_fatal_injury_crashes" true true false 255 Double 0 0,Sum,#,segment_crashes,non_motorized_serious_injuries,-1,-1,segment_crashes,TotalNonMotoristsFatalities,-1,-1', 
                           "INTERSECT", None, '')

In [ ]:
# Impaired Crashes

# Set the query expression
query_expression = "AlcoholInvolvementFlag = 1 OR DrugInvolvementFlag = 1"

# Select features that satisfy the query expression in segment and junction crashes
segment_crashes_selected = arcpy.Select_analysis("segment_crashes_1", "segment_crashes_selected", query_expression)
junction_crashes_selected = arcpy.Select_analysis("junction_crashes", "junction_crashes_selected", query_expression)


# Spatial join to get impaired crashes
arcpy.analysis.SpatialJoin("segments_1", "segment_crashes_selected", r"safety_layers.gdb\segments_2", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,segments_1,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,segments_1,TARGET_FID,-1,-1;route_id "Route ID" true true false 32 Text 0 0,First,#,segments_1,route_id,0,32;functional_classification_code "Functional Classification Code" true true false 2 Short 0 0,First,#,segments_1,functional_classification_code,-1,-1;functional_classification "Functional Classification" true true false 255 Text 0 0,First,#,segments_1,functional_classification,0,255;crashes "crashes" true true false 8 Double 0 0,First,#,segments_1,crashes,-1,-1;injury_crashes "injury_crashes" true true false 8 Double 0 0,First,#,segments_1,injury_crashes,-1,-1;fatal_crashes "fatal_crashes" true true false 8 Double 0 0,First,#,segments_1,fatal_crashes,-1,-1;fatal_injury_crashes "fatal_injury_crashes" true true false 8 Double 0 0,First,#,segments_1,fatal_injury_crashes,-1,-1;non_motorized_fatal_injury_crashes "non_motorized_fatal_injury_crashes" true true false 8 Double 0 0,First,#,segments_1,non_motorized_fatal_injury_crashes,-1,-1;impaired_driving_crashes "impaired_driving_crashes" true true false 255 Double 0 0,Count,#,segment_crashes_selected,TotalUnits,-1,-1', "WITHIN_A_DISTANCE", "150 Feet", '')

arcpy.analysis.SpatialJoin("junctions_1", "junction_crashes_selected", r"safety_layers.gdb\junctions_2", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,junctions_1,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,junctions_1,TARGET_FID,-1,-1;FID_Junctions_SunCloud_Select "FID_Junctions_SunCloud_Select" true true false 4 Long 0 0,First,#,junctions_1,FID_Junctions_SunCloud_Select,-1,-1;FromDate "FromDate" true true false 8 Date 0 0,First,#,junctions_1,FromDate,-1,-1;ToDate "ToDate" true true false 8 Date 0 0,First,#,junctions_1,ToDate,-1,-1;RouteID "RouteID" true true false 32 Text 0 0,First,#,junctions_1,RouteID,0,32;Measure "Measure" true true false 8 Double 0 0,First,#,junctions_1,Measure,-1,-1;MasterJunc "MasterJunc" true true false 38 Text 0 0,First,#,junctions_1,MasterJunc,0,38;NumberOfLe "NumberOfLe" true true false 4 Long 0 0,First,#,junctions_1,NumberOfLe,-1,-1;JunctionGe "JunctionGe" true true false 4 Long 0 0,First,#,junctions_1,JunctionGe,-1,-1;JunctionsS "JunctionsS" true true false 2 Text 0 0,First,#,junctions_1,JunctionsS,0,2;TrafficCon "TrafficCon" true true false 4 Long 0 0,First,#,junctions_1,TrafficCon,-1,-1;TurnCode "TurnCode" true true false 10 Text 0 0,First,#,junctions_1,TurnCode,0,10;Detection "Detection" true true false 4 Long 0 0,First,#,junctions_1,Detection,-1,-1;Signalizat "Signalizat" true true false 4 Long 0 0,First,#,junctions_1,Signalizat,-1,-1;RailCrossi "RailCrossi" true true false 7 Text 0 0,First,#,junctions_1,RailCrossi,0,7;SchoolZone "SchoolZone" true true false 4 Long 0 0,First,#,junctions_1,SchoolZone,-1,-1;BicycleFac "BicycleFac" true true false 4 Long 0 0,First,#,junctions_1,BicycleFac,-1,-1;Crosswalk "Crosswalk" true true false 4 Long 0 0,First,#,junctions_1,Crosswalk,-1,-1;PctGreenTi "PctGreenTi" true true false 4 Long 0 0,First,#,junctions_1,PctGreenTi,-1,-1;PctGreen_1 "PctGreen_1" true true false 4 Long 0 0,First,#,junctions_1,PctGreen_1,-1,-1;SourceID "SourceID" true true false 4 Long 0 0,First,#,junctions_1,SourceID,-1,-1;JunctionMa "JunctionMa" true true false 4 Long 0 0,First,#,junctions_1,JunctionMa,-1,-1;JunctionTy "JunctionTy" true true false 4 Long 0 0,First,#,junctions_1,JunctionTy,-1,-1;OffsetInte "OffsetInte" true true false 1 Text 0 0,First,#,junctions_1,OffsetInte,0,1;OffsetIn_1 "OffsetIn_1" true true false 8 Double 0 0,First,#,junctions_1,OffsetIn_1,-1,-1;InServiceD "InServiceD" true true false 8 Date 0 0,First,#,junctions_1,InServiceD,-1,-1;FID_sun_cloud_routes "FID_sun_cloud_routes" true true false 4 Long 0 0,First,#,junctions_1,FID_sun_cloud_routes,-1,-1;route_id "route_id" true true false 32 Text 0 0,First,#,junctions_1,route_id,0,32;from_measu "from_measu" true true false 8 Double 0 0,First,#,junctions_1,from_measu,-1,-1;to_measure "to_measure" true true false 8 Double 0 0,First,#,junctions_1,to_measure,-1,-1;functional "functional" true true false 4 Long 0 0,First,#,junctions_1,functional,-1,-1;facility_t "facility_t" true true false 4 Long 0 0,First,#,junctions_1,facility_t,-1,-1;crashes "crashes" true true false 255 Text 0 0,First,#,junctions_1,crashes,0,255;injury_crashes "injury_crashes" true true false 8 Double 0 0,First,#,junctions_1,injury_crashes,-1,-1;fatal_crashes "fatal_crashes" true true false 8 Double 0 0,First,#,junctions_1,fatal_crashes,-1,-1;fatal_injury_crashes "fatal_injury_crashes" true true false 8 Double 0 0,First,#,junctions_1,fatal_injury_crashes,-1,-1;non_motorized_fatal_injury "non_motorized_fatal_injury" true true false 8 Double 0 0,First,#,junctions_1,non_motorized_fatal_injury,-1,-1;impaired_driving_crashes "impaired_driving_crashes" true true false 255 Double 0 0,Count,#,junction_crashes_selected,TotalUnits,-1,-1', "WITHIN_A_DISTANCE", "150 Feet", '')


#### Crash Rates

Junction and Segment Crash Rates Calculation: https://safety.fhwa.dot.gov/local_rural/training/fhwasa1210/s3.cfm

In [ ]:
# Calculate length of segments
arcpy.management.CalculateGeometryAttributes("segments_2", 
                                             "Length_Miles LENGTH_GEODESIC", 
                                             "MILES_INT", 
                                             '', 
                                             None, 
                                             "SAME_AS_INPUT")

In [ ]:
# Calculate VMT and MVM for Segments
# Join AADT values to segment layer
# This is buffered 200 feet for instances where there is a dual carriage
arcpy.analysis.SpatialJoin("segments_2", "aadt", r"safety_layers.gdb\segments_3", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,segments_2,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,segments_2,TARGET_FID,-1,-1;Join_Count_1 "Join_Count" true true false 4 Long 0 0,First,#,segments_2,Join_Count_1,-1,-1;TARGET_FID_1 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_2,TARGET_FID_1,-1,-1;route_id "Route ID" true true false 32 Text 0 0,First,#,segments_2,route_id,0,32;functional_classification_code "Functional Classification Code" true true false 2 Short 0 0,First,#,segments_2,functional_classification_code,-1,-1;functional_classification "Functional Classification" true true false 255 Text 0 0,First,#,segments_2,functional_classification,0,255;crashes "crashes" true true false 8 Double 0 0,First,#,segments_2,crashes,-1,-1;injury_crashes "injury_crashes" true true false 8 Double 0 0,First,#,segments_2,injury_crashes,-1,-1;fatal_crashes "fatal_crashes" true true false 8 Double 0 0,First,#,segments_2,fatal_crashes,-1,-1;fatal_injury_crashes "fatal_injury_crashes" true true false 8 Double 0 0,First,#,segments_2,fatal_injury_crashes,-1,-1;non_motorized_fatal_injury_crashes "non_motorized_fatal_injury_crashes" true true false 8 Double 0 0,First,#,segments_2,non_motorized_fatal_injury_crashes,-1,-1;impaired_driving_crashes "impaired_driving_crashes" true true false 8 Double 0 0,First,#,segments_2,impaired_driving_crashes,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,segments_2,Shape_Length,-1,-1;Length_Miles "Length_Miles" true true false 8 Double 0 0,First,#,segments_2,Length_Miles,-1,-1;FromDate "FromDate" true true false 8 Date 0 0,First,#,aadt,FromDate,-1,-1;ToDate "ToDate" true true false 8 Date 0 0,First,#,aadt,ToDate,-1,-1;RouteId "RouteId" true true false 32 Text 0 0,First,#,aadt,RouteId,0,32;FromMeasure "FromMeasure" true true false 8 Double 0 0,First,#,aadt,FromMeasure,-1,-1;ToMeasure "ToMeasure" true true false 8 Double 0 0,First,#,aadt,ToMeasure,-1,-1;AADT "AADT" true true false 4 Long 0 0,First,#,aadt,AADT,-1,-1;InServiceDate "InServiceDate" true true false 8 Date 0 0,First,#,aadt,InServiceDate,-1,-1;Shape_Length_1 "Shape_Length" false true true 8 Double 0 0,First,#,aadt,Shape_Length,-1,-1', "LARGEST_OVERLAP", "200 Feet", '')
arcpy.management.CalculateField("segments_3", "VMT", "!AADT! * !Length_Miles!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

arcpy.management.CalculateField("segments_3", "MVM", "(!VMT! * 365 * 5) / 1000000", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")


Calculate TEV and MEV for Junctions

In [ ]:
# Calculate TEV and MEV for Junctions

# Join AADT values to junction layer
## Average number of vehicles entering intersection can be calculated by adding AADTs for all of the intersection legs, 
## and then dividing that by 4. This assumes that directional split of the roadway for the average day is 50/50 per approach.
arcpy.analysis.SpatialJoin("junctions_2", "aadt", r"safety_layers.gdb\junctions_3", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,junctions_2,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,junctions_2,TARGET_FID,-1,-1;Join_Count_1 "Join_Count" true true false 4 Long 0 0,First,#,junctions_2,Join_Count_1,-1,-1;TARGET_FID_1 "TARGET_FID" true true false 4 Long 0 0,First,#,junctions_2,TARGET_FID_1,-1,-1;FID_Junctions_SunCloud_Select "FID_Junctions_SunCloud_Select" true true false 4 Long 0 0,First,#,junctions_2,FID_Junctions_SunCloud_Select,-1,-1;FromDate "FromDate" true true false 8 Date 0 0,First,#,junctions_2,FromDate,-1,-1;ToDate "ToDate" true true false 8 Date 0 0,First,#,junctions_2,ToDate,-1,-1;RouteID "RouteID" true true false 32 Text 0 0,First,#,junctions_2,RouteID,0,32;Measure "Measure" true true false 8 Double 0 0,First,#,junctions_2,Measure,-1,-1;MasterJunc "MasterJunc" true true false 38 Text 0 0,First,#,junctions_2,MasterJunc,0,38;NumberOfLe "NumberOfLe" true true false 4 Long 0 0,First,#,junctions_2,NumberOfLe,-1,-1;JunctionGe "JunctionGe" true true false 4 Long 0 0,First,#,junctions_2,JunctionGe,-1,-1;JunctionsS "JunctionsS" true true false 2 Text 0 0,First,#,junctions_2,JunctionsS,0,2;TrafficCon "TrafficCon" true true false 4 Long 0 0,First,#,junctions_2,TrafficCon,-1,-1;TurnCode "TurnCode" true true false 10 Text 0 0,First,#,junctions_2,TurnCode,0,10;Detection "Detection" true true false 4 Long 0 0,First,#,junctions_2,Detection,-1,-1;Signalizat "Signalizat" true true false 4 Long 0 0,First,#,junctions_2,Signalizat,-1,-1;RailCrossi "RailCrossi" true true false 7 Text 0 0,First,#,junctions_2,RailCrossi,0,7;SchoolZone "SchoolZone" true true false 4 Long 0 0,First,#,junctions_2,SchoolZone,-1,-1;BicycleFac "BicycleFac" true true false 4 Long 0 0,First,#,junctions_2,BicycleFac,-1,-1;Crosswalk "Crosswalk" true true false 4 Long 0 0,First,#,junctions_2,Crosswalk,-1,-1;PctGreenTi "PctGreenTi" true true false 4 Long 0 0,First,#,junctions_2,PctGreenTi,-1,-1;PctGreen_1 "PctGreen_1" true true false 4 Long 0 0,First,#,junctions_2,PctGreen_1,-1,-1;SourceID "SourceID" true true false 4 Long 0 0,First,#,junctions_2,SourceID,-1,-1;JunctionMa "JunctionMa" true true false 4 Long 0 0,First,#,junctions_2,JunctionMa,-1,-1;JunctionTy "JunctionTy" true true false 4 Long 0 0,First,#,junctions_2,JunctionTy,-1,-1;OffsetInte "OffsetInte" true true false 1 Text 0 0,First,#,junctions_2,OffsetInte,0,1;OffsetIn_1 "OffsetIn_1" true true false 8 Double 0 0,First,#,junctions_2,OffsetIn_1,-1,-1;InServiceD "InServiceD" true true false 8 Date 0 0,First,#,junctions_2,InServiceD,-1,-1;FID_sun_cloud_routes "FID_sun_cloud_routes" true true false 4 Long 0 0,First,#,junctions_2,FID_sun_cloud_routes,-1,-1;route_id "route_id" true true false 32 Text 0 0,First,#,junctions_2,route_id,0,32;from_measu "from_measu" true true false 8 Double 0 0,First,#,junctions_2,from_measu,-1,-1;to_measure "to_measure" true true false 8 Double 0 0,First,#,junctions_2,to_measure,-1,-1;functional "functional" true true false 4 Long 0 0,First,#,junctions_2,functional,-1,-1;facility_t "facility_t" true true false 4 Long 0 0,First,#,junctions_2,facility_t,-1,-1;crashes "crashes" true true false 255 Text 0 0,First,#,junctions_2,crashes,0,255;injury_crashes "injury_crashes" true true false 8 Double 0 0,First,#,junctions_2,injury_crashes,-1,-1;fatal_crashes "fatal_crashes" true true false 8 Double 0 0,First,#,junctions_2,fatal_crashes,-1,-1;fatal_injury_crashes "fatal_injury_crashes" true true false 8 Double 0 0,First,#,junctions_2,fatal_injury_crashes,-1,-1;non_motorized_fatal_injury "non_motorized_fatal_injury" true true false 8 Double 0 0,First,#,junctions_2,non_motorized_fatal_injury,-1,-1;impaired_driving_crashes "impaired_driving_crashes" true true false 8 Double 0 0,First,#,junctions_2,impaired_driving_crashes,-1,-1;AADT "AADT" true true false 255 Double 0 0,Sum,#,aadt,AADT,-1,-1;aadt_major "aadt_major" true true false 255 Double 0 0,Max,#,aadt,AADT,-1,-1;aadt_minor "aadt_minor" true true false 255 Double 0 0,Min,#,aadt,AADT,-1,-1', "WITHIN_A_DISTANCE", "150 Feet", '')

arcpy.management.CalculateField("junctions_3", 
                                "TEV", 
                                "!AADT! / 4", 
                                "PYTHON3", 
                                '', 
                                "DOUBLE", 
                                "NO_ENFORCE_DOMAINS")

arcpy.management.CalculateField("junctions_3", 
                                "MEV", 
                                "(!TEV! * 365 * 5) / 1000000", 
                                "PYTHON3", 
                                '', 
                                "DOUBLE", 
                                "NO_ENFORCE_DOMAINS")

In [ ]:
# Junctions Crash Rate
# Injury Rate
arcpy.management.CalculateField("junctions_3", 
                                "injury_rate", 
                                "!injury_crashes! / !MEV!", 
                                "PYTHON3", 
                                '', 
                                "DOUBLE", 
                                "NO_ENFORCE_DOMAINS")

# Fatal Injury Rate
arcpy.management.CalculateField("junctions_3", 
                                "fatal_injury_rate", 
                                "!fatal_injury_crashes! / !MEV!", 
                                "PYTHON3", 
                                '', 
                                "DOUBLE", 
                                "NO_ENFORCE_DOMAINS")


In [ ]:
# Segment Crash Rate
# Injury Rate
arcpy.management.CalculateField("segments_3", 
                                "injury_rate", 
                                "0 if !MVM! == 0 else !injury_crashes! / !MVM!", 
                                "PYTHON3", 
                                '', 
                                "DOUBLE", 
                                "NO_ENFORCE_DOMAINS")


# Fatal Injury Rate
arcpy.management.CalculateField("segments_3", 
                                "fatal_injuy_rate", 
                                "0 if !MVM! == 0 else !fatal_injury_crashes! / !MVM!", 
                                "PYTHON3", 
                                '', 
                                "DOUBLE", 
                                "NO_ENFORCE_DOMAINS")


#### Predictive Crashes Calculation

In [ ]:
# Spatially join urban area info to segment layer

# Perform the spatial join
arcpy.analysis.SpatialJoin("segments_3", "urban_areas", r"safety_layers.gdb\segments_4", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,segments_3,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,segments_3,TARGET_FID,-1,-1;Join_Count_1 "Join_Count" true true false 4 Long 0 0,First,#,segments_3,Join_Count_1,-1,-1;TARGET_FID_1 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_3,TARGET_FID_1,-1,-1;Join_Count_12 "Join_Count" true true false 4 Long 0 0,First,#,segments_3,Join_Count_12,-1,-1;TARGET_FID_12 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_3,TARGET_FID_12,-1,-1;route_id "Route ID" true true false 32 Text 0 0,First,#,segments_3,route_id,0,32;functional_classification_code "Functional Classification Code" true true false 2 Short 0 0,First,#,segments_3,functional_classification_code,-1,-1;functional_classification "Functional Classification" true true false 255 Text 0 0,First,#,segments_3,functional_classification,0,255;crashes "crashes" true true false 8 Double 0 0,First,#,segments_3,crashes,-1,-1;injury_crashes "injury_crashes" true true false 8 Double 0 0,First,#,segments_3,injury_crashes,-1,-1;fatal_crashes "fatal_crashes" true true false 8 Double 0 0,First,#,segments_3,fatal_crashes,-1,-1;fatal_injury_crashes "fatal_injury_crashes" true true false 8 Double 0 0,First,#,segments_3,fatal_injury_crashes,-1,-1;non_motorized_fatal_injury_crashes "non_motorized_fatal_injury_crashes" true true false 8 Double 0 0,First,#,segments_3,non_motorized_fatal_injury_crashes,-1,-1;impaired_driving_crashes "impaired_driving_crashes" true true false 8 Double 0 0,First,#,segments_3,impaired_driving_crashes,-1,-1;Length_Miles "Length_Miles" true true false 8 Double 0 0,First,#,segments_3,Length_Miles,-1,-1;FromDate "FromDate" true true false 8 Date 0 0,First,#,segments_3,FromDate,-1,-1;ToDate "ToDate" true true false 8 Date 0 0,First,#,segments_3,ToDate,-1,-1;RouteId "RouteId" true true false 32 Text 0 0,First,#,segments_3,RouteId,0,32;FromMeasure "FromMeasure" true true false 8 Double 0 0,First,#,segments_3,FromMeasure,-1,-1;ToMeasure "ToMeasure" true true false 8 Double 0 0,First,#,segments_3,ToMeasure,-1,-1;AADT "AADT" true true false 4 Long 0 0,First,#,segments_3,AADT,-1,-1;InServiceDate "InServiceDate" true true false 8 Date 0 0,First,#,segments_3,InServiceDate,-1,-1;Shape_Length_1 "Shape_Length" true true false 8 Double 0 0,First,#,segments_3,Shape_Length_1,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,segments_3,Shape_Length,-1,-1;VMT "VMT" true true false 8 Double 0 0,First,#,segments_3,VMT,-1,-1;MVM "MVM" true true false 8 Double 0 0,First,#,segments_3,MVM,-1,-1;injury_rate "injury_rate" true true false 8 Double 0 0,First,#,segments_3,injury_rate,-1,-1;fatal_injuy_rate "fatal_injuy_rate" true true false 8 Double 0 0,First,#,segments_3,fatal_injuy_rate,-1,-1;UA_ID "URBAN AREA ID" true true false 5 Text 0 0,First,#,urban_areas,UA_ID,0,5;NAME "NAME" true true false 60 Text 0 0,First,#,urban_areas,NAME,0,60;LSAD "LEGAL/STATISTICAL AREA CODE (LSAD)" true true false 2 Text 0 0,First,#,urban_areas,LSAD,0,2;LSAD_DESC "LSAD DESCRIPTION" true true false 15 Text 0 0,First,#,urban_areas,LSAD_DESC,0,15;POP2010 "POP2010" true true false 4 Long 0 0,First,#,urban_areas,POP2010,-1,-1;POP10_SQMI "POP10_SQMI" true true false 8 Double 0 0,First,#,urban_areas,POP10_SQMI,-1,-1;HSE_UNITS "HOUSING UNITS" true true false 4 Long 0 0,First,#,urban_areas,HSE_UNITS,-1,-1;SQMI "SQMI" true true false 8 Double 0 0,First,#,urban_areas,SQMI,-1,-1;Shape_Length_12 "Shape_Length" false true true 8 Double 0 0,First,#,urban_areas,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,urban_areas,Shape_Area,-1,-1', "WITHIN", None, '')


In [ ]:
arcpy.management.CalculateField("segments_4", "in_urban_area", "urban_area(!POP2010!)", "PYTHON3", """def urban_area(pop):
    if pop >= 0:
        return 1
    else:
        return 0""", "DOUBLE", "NO_ENFORCE_DOMAINS")

In [ ]:
# Join ATIS layer to get divided/undivided attributes
arcpy.analysis.SpatialJoin("segments_4", "atis", r"safety_final.gdb\segments_5", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,segments_4,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,segments_4,TARGET_FID,-1,-1;Join_Count_1 "Join_Count" true true false 4 Long 0 0,First,#,segments_4,Join_Count_1,-1,-1;TARGET_FID_1 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_4,TARGET_FID_1,-1,-1;Join_Count_12 "Join_Count" true true false 4 Long 0 0,First,#,segments_4,Join_Count_12,-1,-1;TARGET_FID_12 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_4,TARGET_FID_12,-1,-1;Join_Count_12_13 "Join_Count" true true false 4 Long 0 0,First,#,segments_4,Join_Count_12_13,-1,-1;TARGET_FID_12_13 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_4,TARGET_FID_12_13,-1,-1;route_id "Route ID" true true false 32 Text 0 0,First,#,segments_4,route_id,0,32;functional_classification_code "Functional Classification Code" true true false 2 Short 0 0,First,#,segments_4,functional_classification_code,-1,-1;functional_classification "Functional Classification" true true false 255 Text 0 0,First,#,segments_4,functional_classification,0,255;crashes "crashes" true true false 8 Double 0 0,First,#,segments_4,crashes,-1,-1;injury_crashes "injury_crashes" true true false 8 Double 0 0,First,#,segments_4,injury_crashes,-1,-1;fatal_crashes "fatal_crashes" true true false 8 Double 0 0,First,#,segments_4,fatal_crashes,-1,-1;fatal_injury_crashes "fatal_injury_crashes" true true false 8 Double 0 0,First,#,segments_4,fatal_injury_crashes,-1,-1;non_motorized_fatal_injury_crashes "non_motorized_fatal_injury_crashes" true true false 8 Double 0 0,First,#,segments_4,non_motorized_fatal_injury_crashes,-1,-1;impaired_driving_crashes "impaired_driving_crashes" true true false 8 Double 0 0,First,#,segments_4,impaired_driving_crashes,-1,-1;Length_Miles "Length_Miles" true true false 8 Double 0 0,First,#,segments_4,Length_Miles,-1,-1;FromDate "FromDate" true true false 8 Date 0 0,First,#,segments_4,FromDate,-1,-1;ToDate "ToDate" true true false 8 Date 0 0,First,#,segments_4,ToDate,-1,-1;RouteId "RouteId" true true false 32 Text 0 0,First,#,segments_4,RouteId,0,32;FromMeasure "FromMeasure" true true false 8 Double 0 0,First,#,segments_4,FromMeasure,-1,-1;ToMeasure "ToMeasure" true true false 8 Double 0 0,First,#,segments_4,ToMeasure,-1,-1;AADT "AADT" true true false 4 Long 0 0,First,#,segments_4,AADT,-1,-1;InServiceDate "InServiceDate" true true false 8 Date 0 0,First,#,segments_4,InServiceDate,-1,-1;Shape_Length_1 "Shape_Length" true true false 8 Double 0 0,First,#,segments_4,Shape_Length_1,-1,-1;VMT "VMT" true true false 8 Double 0 0,First,#,segments_4,VMT,-1,-1;MVM "MVM" true true false 8 Double 0 0,First,#,segments_4,MVM,-1,-1;injury_rate "injury_rate" true true false 8 Double 0 0,First,#,segments_4,injury_rate,-1,-1;fatal_injuy_rate "fatal_injuy_rate" true true false 8 Double 0 0,First,#,segments_4,fatal_injuy_rate,-1,-1;UA_ID "URBAN AREA ID" true true false 5 Text 0 0,First,#,segments_4,UA_ID,0,5;NAME "NAME" true true false 60 Text 0 0,First,#,segments_4,NAME,0,60;LSAD "LEGAL/STATISTICAL AREA CODE (LSAD)" true true false 2 Text 0 0,First,#,segments_4,LSAD,0,2;LSAD_DESC "LSAD DESCRIPTION" true true false 15 Text 0 0,First,#,segments_4,LSAD_DESC,0,15;POP2010 "POP2010" true true false 4 Long 0 0,First,#,segments_4,POP2010,-1,-1;POP10_SQMI "POP10_SQMI" true true false 8 Double 0 0,First,#,segments_4,POP10_SQMI,-1,-1;HSE_UNITS "HOUSING UNITS" true true false 4 Long 0 0,First,#,segments_4,HSE_UNITS,-1,-1;SQMI "SQMI" true true false 8 Double 0 0,First,#,segments_4,SQMI,-1,-1;Shape_Length_12 "Shape_Length" true true false 8 Double 0 0,First,#,segments_4,Shape_Length_12,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,segments_4,Shape_Length,-1,-1;in_urban_area "in_urban_area" true true false 8 Double 0 0,First,#,segments_4,in_urban_area,-1,-1;RouteIsDivided "Route Is Divided" true true false 2 Short 0 0,First,#,atis,RouteIsDivided,-1,-1', "LARGEST_OVERLAP", None, '')

# Fill in missing median data
arcpy.analysis.SpatialJoin("segments_5", "median", r"safety_layers.gdb\segments_6", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,segments_5,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,segments_5,TARGET_FID,-1,-1;Join_Count_1 "Join_Count" true true false 4 Long 0 0,First,#,segments_5,Join_Count_1,-1,-1;TARGET_FID_1 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_5,TARGET_FID_1,-1,-1;Join_Count_12 "Join_Count" true true false 4 Long 0 0,First,#,segments_5,Join_Count_12,-1,-1;TARGET_FID_12 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_5,TARGET_FID_12,-1,-1;Join_Count_12_13 "Join_Count" true true false 4 Long 0 0,First,#,segments_5,Join_Count_12_13,-1,-1;TARGET_FID_12_13 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_5,TARGET_FID_12_13,-1,-1;Join_Count_12_13_14 "Join_Count" true true false 4 Long 0 0,First,#,segments_5,Join_Count_12_13_14,-1,-1;TARGET_FID_12_13_14 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_5,TARGET_FID_12_13_14,-1,-1;route_id "Route ID" true true false 32 Text 0 0,First,#,segments_5,route_id,0,32;functional_classification_code "Functional Classification Code" true true false 2 Short 0 0,First,#,segments_5,functional_classification_code,-1,-1;functional_classification "Functional Classification" true true false 255 Text 0 0,First,#,segments_5,functional_classification,0,255;crashes "crashes" true true false 8 Double 0 0,First,#,segments_5,crashes,-1,-1;injury_crashes "injury_crashes" true true false 8 Double 0 0,First,#,segments_5,injury_crashes,-1,-1;fatal_crashes "fatal_crashes" true true false 8 Double 0 0,First,#,segments_5,fatal_crashes,-1,-1;fatal_injury_crashes "fatal_injury_crashes" true true false 8 Double 0 0,First,#,segments_5,fatal_injury_crashes,-1,-1;non_motorized_fatal_injury_crashes "non_motorized_fatal_injury_crashes" true true false 8 Double 0 0,First,#,segments_5,non_motorized_fatal_injury_crashes,-1,-1;impaired_driving_crashes "impaired_driving_crashes" true true false 8 Double 0 0,First,#,segments_5,impaired_driving_crashes,-1,-1;Length_Miles "Length_Miles" true true false 8 Double 0 0,First,#,segments_5,Length_Miles,-1,-1;FromDate "FromDate" true true false 8 Date 0 0,First,#,segments_5,FromDate,-1,-1;ToDate "ToDate" true true false 8 Date 0 0,First,#,segments_5,ToDate,-1,-1;RouteId "RouteId" true true false 32 Text 0 0,First,#,segments_5,RouteId,0,32;FromMeasure "FromMeasure" true true false 8 Double 0 0,First,#,segments_5,FromMeasure,-1,-1;ToMeasure "ToMeasure" true true false 8 Double 0 0,First,#,segments_5,ToMeasure,-1,-1;AADT "AADT" true true false 4 Long 0 0,First,#,segments_5,AADT,-1,-1;InServiceDate "InServiceDate" true true false 8 Date 0 0,First,#,segments_5,InServiceDate,-1,-1;Shape_Length_1 "Shape_Length" true true false 8 Double 0 0,First,#,segments_5,Shape_Length_1,-1,-1;VMT "VMT" true true false 8 Double 0 0,First,#,segments_5,VMT,-1,-1;MVM "MVM" true true false 8 Double 0 0,First,#,segments_5,MVM,-1,-1;injury_rate "injury_rate" true true false 8 Double 0 0,First,#,segments_5,injury_rate,-1,-1;fatal_injuy_rate "fatal_injuy_rate" true true false 8 Double 0 0,First,#,segments_5,fatal_injuy_rate,-1,-1;UA_ID "URBAN AREA ID" true true false 5 Text 0 0,First,#,segments_5,UA_ID,0,5;NAME "NAME" true true false 60 Text 0 0,First,#,segments_5,NAME,0,60;LSAD "LEGAL/STATISTICAL AREA CODE (LSAD)" true true false 2 Text 0 0,First,#,segments_5,LSAD,0,2;LSAD_DESC "LSAD DESCRIPTION" true true false 15 Text 0 0,First,#,segments_5,LSAD_DESC,0,15;POP2010 "POP2010" true true false 4 Long 0 0,First,#,segments_5,POP2010,-1,-1;POP10_SQMI "POP10_SQMI" true true false 8 Double 0 0,First,#,segments_5,POP10_SQMI,-1,-1;HSE_UNITS "HOUSING UNITS" true true false 4 Long 0 0,First,#,segments_5,HSE_UNITS,-1,-1;SQMI "SQMI" true true false 8 Double 0 0,First,#,segments_5,SQMI,-1,-1;Shape_Length_12 "Shape_Length" true true false 8 Double 0 0,First,#,segments_5,Shape_Length_12,-1,-1;in_urban_area "in_urban_area" true true false 8 Double 0 0,First,#,segments_5,in_urban_area,-1,-1;RouteIsDivided "Route Is Divided" true true false 2 Short 0 0,First,#,segments_5,RouteIsDivided,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,segments_5,Shape_Length,-1,-1;FromDate_1 "FromDate" true true false 200 Text 0 0,First,#,median,FromDate,0,200;ToDate_1 "ToDate" true true false 200 Text 0 0,First,#,median,ToDate,0,200;RouteId_1 "RouteId" true true false 200 Text 0 0,First,#,median,RouteId,0,200;FromMeasure_1 "FromMeasure" true true false 200 Text 0 0,First,#,median,FromMeasure,0,200;ToMeasure_1 "ToMeasure" true true false 200 Text 0 0,First,#,median,ToMeasure,0,200;MedianWidthFt "MedianWidthFt" true true false 200 Text 0 0,First,#,median,MedianWidthFt,0,200;MedianType "MedianType" true true false 200 Text 0 0,First,#,median,MedianType,0,200;NumberOfLanes "NumberOfLanes" true true false 200 Text 0 0,First,#,median,NumberOfLanes,0,200;BeginWidthFt "BeginWidthFt" true true false 200 Text 0 0,First,#,median,BeginWidthFt,0,200;EndWidthFt "EndWidthFt" true true false 200 Text 0 0,First,#,median,EndWidthFt,0,200;OTTYear "OTTYear" true true false 200 Text 0 0,First,#,median,OTTYear,0,200;AuxLaneType "AuxLaneType" true true false 200 Text 0 0,First,#,median,AuxLaneType,0,200;AuxLaneTypeForSymbol "AuxLaneTypeForSymbol" true true false 200 Text 0 0,First,#,median,AuxLaneTypeForSymbol,0,200;InServiceDate_1 "InServiceDate" true true false 200 Text 0 0,First,#,median,InServiceDate,0,200;OutServiceDate "OutServiceDate" true true false 200 Text 0 0,First,#,median,OutServiceDate,0,200;SourceYear "SourceYear" true true false 200 Text 0 0,First,#,median,SourceYear,0,200;GlobalID "GlobalID" false false true 38 GlobalID 0 0,First,#,median,GlobalID,-1,-1;Shape_Length_12_13 "Shape_Length" false true true 8 Double 0 0,First,#,median,Shape_Length,-1,-1', "LARGEST_OVERLAP", None, '')

# Filter to just divided/undivided null and median not null
arcpy.management.CalculateField("segments_6", "divided", 
    "choose_spf(!divided!, !MedianType!, !RouteIsDivided!)", "PYTHON3", """
def choose_spf(divided, median_type, route_is_divided):
    if median_type == "Curbed - raised" or median_type == "Positive barrier - rigid" or route_is_divided == 1:
        return 1
    else:
        return 0
""", "DOUBLE")

# Join HPMS to segments to get Number of Lanes
arcpy.analysis.SpatialJoin("segments_6", "through_lanes", r"safety_layers.gdb\segments_7", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,segments_6,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,segments_6,TARGET_FID,-1,-1;Join_Count_1 "Join_Count" true true false 4 Long 0 0,First,#,segments_6,Join_Count_1,-1,-1;TARGET_FID_1 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_6,TARGET_FID_1,-1,-1;Join_Count_12 "Join_Count" true true false 4 Long 0 0,First,#,segments_6,Join_Count_12,-1,-1;TARGET_FID_12 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_6,TARGET_FID_12,-1,-1;Join_Count_12_13 "Join_Count" true true false 4 Long 0 0,First,#,segments_6,Join_Count_12_13,-1,-1;TARGET_FID_12_13 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_6,TARGET_FID_12_13,-1,-1;Join_Count_12_13_14 "Join_Count" true true false 4 Long 0 0,First,#,segments_6,Join_Count_12_13_14,-1,-1;TARGET_FID_12_13_14 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_6,TARGET_FID_12_13_14,-1,-1;Join_Count_12_13_14_15 "Join_Count" true true false 4 Long 0 0,First,#,segments_6,Join_Count_12_13_14_15,-1,-1;TARGET_FID_12_13_14_15 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_6,TARGET_FID_12_13_14_15,-1,-1;route_id "Route ID" true true false 32 Text 0 0,First,#,segments_6,route_id,0,32;functional_classification_code "Functional Classification Code" true true false 2 Short 0 0,First,#,segments_6,functional_classification_code,-1,-1;functional_classification "Functional Classification" true true false 255 Text 0 0,First,#,segments_6,functional_classification,0,255;crashes "crashes" true true false 8 Double 0 0,First,#,segments_6,crashes,-1,-1;injury_crashes "injury_crashes" true true false 8 Double 0 0,First,#,segments_6,injury_crashes,-1,-1;fatal_crashes "fatal_crashes" true true false 8 Double 0 0,First,#,segments_6,fatal_crashes,-1,-1;fatal_injury_crashes "fatal_injury_crashes" true true false 8 Double 0 0,First,#,segments_6,fatal_injury_crashes,-1,-1;non_motorized_fatal_injury_crashes "non_motorized_fatal_injury_crashes" true true false 8 Double 0 0,First,#,segments_6,non_motorized_fatal_injury_crashes,-1,-1;impaired_driving_crashes "impaired_driving_crashes" true true false 8 Double 0 0,First,#,segments_6,impaired_driving_crashes,-1,-1;Length_Miles "Length_Miles" true true false 8 Double 0 0,First,#,segments_6,Length_Miles,-1,-1;FromDate "FromDate" true true false 8 Date 0 0,First,#,segments_6,FromDate,-1,-1;ToDate "ToDate" true true false 8 Date 0 0,First,#,segments_6,ToDate,-1,-1;RouteId "RouteId" true true false 32 Text 0 0,First,#,segments_6,RouteId,0,32;FromMeasure "FromMeasure" true true false 8 Double 0 0,First,#,segments_6,FromMeasure,-1,-1;ToMeasure "ToMeasure" true true false 8 Double 0 0,First,#,segments_6,ToMeasure,-1,-1;AADT "AADT" true true false 4 Long 0 0,First,#,segments_6,AADT,-1,-1;InServiceDate "InServiceDate" true true false 8 Date 0 0,First,#,segments_6,InServiceDate,-1,-1;Shape_Length_1 "Shape_Length" true true false 8 Double 0 0,First,#,segments_6,Shape_Length_1,-1,-1;VMT "VMT" true true false 8 Double 0 0,First,#,segments_6,VMT,-1,-1;MVM "MVM" true true false 8 Double 0 0,First,#,segments_6,MVM,-1,-1;injury_rate "injury_rate" true true false 8 Double 0 0,First,#,segments_6,injury_rate,-1,-1;fatal_injuy_rate "fatal_injuy_rate" true true false 8 Double 0 0,First,#,segments_6,fatal_injuy_rate,-1,-1;UA_ID "URBAN AREA ID" true true false 5 Text 0 0,First,#,segments_6,UA_ID,0,5;NAME "NAME" true true false 60 Text 0 0,First,#,segments_6,NAME,0,60;LSAD "LEGAL/STATISTICAL AREA CODE (LSAD)" true true false 2 Text 0 0,First,#,segments_6,LSAD,0,2;LSAD_DESC "LSAD DESCRIPTION" true true false 15 Text 0 0,First,#,segments_6,LSAD_DESC,0,15;POP2010 "POP2010" true true false 4 Long 0 0,First,#,segments_6,POP2010,-1,-1;POP10_SQMI "POP10_SQMI" true true false 8 Double 0 0,First,#,segments_6,POP10_SQMI,-1,-1;HSE_UNITS "HOUSING UNITS" true true false 4 Long 0 0,First,#,segments_6,HSE_UNITS,-1,-1;SQMI "SQMI" true true false 8 Double 0 0,First,#,segments_6,SQMI,-1,-1;Shape_Length_12 "Shape_Length" true true false 8 Double 0 0,First,#,segments_6,Shape_Length_12,-1,-1;in_urban_area "in_urban_area" true true false 8 Double 0 0,First,#,segments_6,in_urban_area,-1,-1;RouteIsDivided "Route Is Divided" true true false 2 Short 0 0,First,#,segments_6,RouteIsDivided,-1,-1;FromDate_1 "FromDate" true true false 200 Text 0 0,First,#,segments_6,FromDate_1,0,200;ToDate_1 "ToDate" true true false 200 Text 0 0,First,#,segments_6,ToDate_1,0,200;RouteId_1 "RouteId" true true false 200 Text 0 0,First,#,segments_6,RouteId_1,0,200;FromMeasure_1 "FromMeasure" true true false 200 Text 0 0,First,#,segments_6,FromMeasure_1,0,200;ToMeasure_1 "ToMeasure" true true false 200 Text 0 0,First,#,segments_6,ToMeasure_1,0,200;MedianWidthFt "MedianWidthFt" true true false 200 Text 0 0,First,#,segments_6,MedianWidthFt,0,200;MedianType "MedianType" true true false 200 Text 0 0,First,#,segments_6,MedianType,0,200;NumberOfLanes "NumberOfLanes" true true false 200 Text 0 0,First,#,through_lanes,NumberOfLanes,-1,-1;BeginWidthFt "BeginWidthFt" true true false 200 Text 0 0,First,#,segments_6,BeginWidthFt,0,200;EndWidthFt "EndWidthFt" true true false 200 Text 0 0,First,#,segments_6,EndWidthFt,0,200;OTTYear "OTTYear" true true false 200 Text 0 0,First,#,segments_6,OTTYear,0,200;AuxLaneType "AuxLaneType" true true false 200 Text 0 0,First,#,segments_6,AuxLaneType,0,200;AuxLaneTypeForSymbol "AuxLaneTypeForSymbol" true true false 200 Text 0 0,First,#,segments_6,AuxLaneTypeForSymbol,0,200;InServiceDate_1 "InServiceDate" true true false 200 Text 0 0,First,#,segments_6,InServiceDate_1,0,200;OutServiceDate "OutServiceDate" true true false 200 Text 0 0,First,#,segments_6,OutServiceDate,0,200;SourceYear "SourceYear" true true false 200 Text 0 0,First,#,segments_6,SourceYear,0,200;Shape_Length_12_13 "Shape_Length" true true false 8 Double 0 0,First,#,segments_6,Shape_Length_12_13,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,segments_6,Shape_Length,-1,-1;divided "divided" true true false 8 Double 0 0,First,#,segments_6,divided,-1,-1', "LARGEST_OVERLAP", "150 Feet", '')

In [ ]:
# Change to double
arcpy.management.CalculateField("segments_7", "num_lanes", "!NumberOfLanes!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")


In [ ]:
# Get number of entering lanes by summing number of lanes for each intersecting segment
arcpy.analysis.SpatialJoin("junctions_3", "segments_7", r"safety_layers.gdb\junctions_4", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,junctions_3,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,junctions_3,TARGET_FID,-1,-1;Join_Count_1 "Join_Count" true true false 4 Long 0 0,First,#,junctions_3,Join_Count_1,-1,-1;TARGET_FID_1 "TARGET_FID" true true false 4 Long 0 0,First,#,junctions_3,TARGET_FID_1,-1,-1;Join_Count_12 "Join_Count" true true false 4 Long 0 0,First,#,junctions_3,Join_Count_12,-1,-1;TARGET_FID_12 "TARGET_FID" true true false 4 Long 0 0,First,#,junctions_3,TARGET_FID_12,-1,-1;FID_Junctions_SunCloud_Select "FID_Junctions_SunCloud_Select" true true false 4 Long 0 0,First,#,junctions_3,FID_Junctions_SunCloud_Select,-1,-1;FromDate "FromDate" true true false 8 Date 0 0,First,#,junctions_3,FromDate,-1,-1;ToDate "ToDate" true true false 8 Date 0 0,First,#,junctions_3,ToDate,-1,-1;RouteID "RouteID" true true false 32 Text 0 0,First,#,junctions_3,RouteID,0,32;Measure "Measure" true true false 8 Double 0 0,First,#,junctions_3,Measure,-1,-1;MasterJunc "MasterJunc" true true false 38 Text 0 0,First,#,junctions_3,MasterJunc,0,38;NumberOfLe "NumberOfLe" true true false 4 Long 0 0,First,#,junctions_3,NumberOfLe,-1,-1;JunctionGe "JunctionGe" true true false 4 Long 0 0,First,#,junctions_3,JunctionGe,-1,-1;JunctionsS "JunctionsS" true true false 2 Text 0 0,First,#,junctions_3,JunctionsS,0,2;TrafficCon "TrafficCon" true true false 4 Long 0 0,First,#,junctions_3,TrafficCon,-1,-1;TurnCode "TurnCode" true true false 10 Text 0 0,First,#,junctions_3,TurnCode,0,10;Detection "Detection" true true false 4 Long 0 0,First,#,junctions_3,Detection,-1,-1;Signalizat "Signalizat" true true false 4 Long 0 0,First,#,junctions_3,Signalizat,-1,-1;RailCrossi "RailCrossi" true true false 7 Text 0 0,First,#,junctions_3,RailCrossi,0,7;SchoolZone "SchoolZone" true true false 4 Long 0 0,First,#,junctions_3,SchoolZone,-1,-1;BicycleFac "BicycleFac" true true false 4 Long 0 0,First,#,junctions_3,BicycleFac,-1,-1;Crosswalk "Crosswalk" true true false 4 Long 0 0,First,#,junctions_3,Crosswalk,-1,-1;PctGreenTi "PctGreenTi" true true false 4 Long 0 0,First,#,junctions_3,PctGreenTi,-1,-1;PctGreen_1 "PctGreen_1" true true false 4 Long 0 0,First,#,junctions_3,PctGreen_1,-1,-1;SourceID "SourceID" true true false 4 Long 0 0,First,#,junctions_3,SourceID,-1,-1;JunctionMa "JunctionMa" true true false 4 Long 0 0,First,#,junctions_3,JunctionMa,-1,-1;JunctionTy "JunctionTy" true true false 4 Long 0 0,First,#,junctions_3,JunctionTy,-1,-1;OffsetInte "OffsetInte" true true false 1 Text 0 0,First,#,junctions_3,OffsetInte,0,1;OffsetIn_1 "OffsetIn_1" true true false 8 Double 0 0,First,#,junctions_3,OffsetIn_1,-1,-1;InServiceD "InServiceD" true true false 8 Date 0 0,First,#,junctions_3,InServiceD,-1,-1;FID_sun_cloud_routes "FID_sun_cloud_routes" true true false 4 Long 0 0,First,#,junctions_3,FID_sun_cloud_routes,-1,-1;route_id "route_id" true true false 32 Text 0 0,First,#,junctions_3,route_id,0,32;from_measu "from_measu" true true false 8 Double 0 0,First,#,junctions_3,from_measu,-1,-1;to_measure "to_measure" true true false 8 Double 0 0,First,#,junctions_3,to_measure,-1,-1;functional "functional" true true false 4 Long 0 0,First,#,junctions_3,functional,-1,-1;facility_t "facility_t" true true false 4 Long 0 0,First,#,junctions_3,facility_t,-1,-1;crashes "crashes" true true false 255 Text 0 0,First,#,junctions_3,crashes,0,255;injury_crashes "injury_crashes" true true false 8 Double 0 0,First,#,junctions_3,injury_crashes,-1,-1;fatal_crashes "fatal_crashes" true true false 8 Double 0 0,First,#,junctions_3,fatal_crashes,-1,-1;fatal_injury_crashes "fatal_injury_crashes" true true false 8 Double 0 0,First,#,junctions_3,fatal_injury_crashes,-1,-1;non_motorized_fatal_injury "non_motorized_fatal_injury" true true false 8 Double 0 0,First,#,junctions_3,non_motorized_fatal_injury,-1,-1;impaired_driving_crashes "impaired_driving_crashes" true true false 8 Double 0 0,First,#,junctions_3,impaired_driving_crashes,-1,-1;AADT "AADT" true true false 8 Double 0 0,First,#,junctions_3,AADT,-1,-1;aadt_major "aadt_major" true true false 8 Double 0 0,First,#,junctions_3,aadt_major,-1,-1;aadt_minor "aadt_minor" true true false 8 Double 0 0,First,#,junctions_3,aadt_minor,-1,-1;TEV "TEV" true true false 8 Double 0 0,First,#,junctions_3,TEV,-1,-1;MEV "MEV" true true false 8 Double 0 0,First,#,junctions_3,MEV,-1,-1;injury_rate "injury_rate" true true false 8 Double 0 0,First,#,junctions_3,injury_rate,-1,-1;fatal_injury_rate "fatal_injury_rate" true true false 8 Double 0 0,First,#,junctions_3,fatal_injury_rate,-1,-1;lanes_entering "lanes_entering" true true false 255 Double 0 0,Sum,#,segments_6,num_lanes,-1,-1', "INTERSECT", "150 Feet", '')


In [ ]:
# QAQC 
# Create a data frame of counts of each filter type for SPFs to check if there are segment or junction outliers

# Set the input layer
junctions_safety = "junctions_4"
segments_safety = "segments_7"

# Convert the input layer to a NumPy array
junctions_array = arcpy.da.TableToNumPyArray(junctions_safety, ["TrafficCon", "NumberOfLe", "lanes_entering"])
segments_array = arcpy.da.TableToNumPyArray(segments_safety, ["num_lanes", "divided", "in_urban_area"])

# Convert the NumPy array to a Pandas dataframe
junctions_df = pd.DataFrame(junctions_array)
segments_df = pd.DataFrame(segments_array)

# Group the dataframe by the values and count the number of rows in each group
junction_counts = junctions_df.groupby(["TrafficCon", "NumberOfLe", "lanes_entering"]).size()
segments_counts = segments_df.groupby(["num_lanes", "divided", "in_urban_area"]).size()

# Print the resulting dataframe
print(junction_counts)
print(segments_counts)

In [ ]:
# QAQC
# Find the number of null values applicable to the SPFs for each layer

# Count the number of null values for each column in the dataframe
junctions_null_counts = junctions_df.isna().sum()
segments_null_counts = segments_df.isna().sum()

# Print the null counts
print("Null counts in junctions:")
print(junctions_null_counts)

print("Null counts in segments:")
print(segments_null_counts)


In [ ]:
# QAQC
# Count the number of zero values for each column in the dataframe
junctions_zero_counts = (junctions_df == 0).sum()
segments_zero_counts = (segments_df == 0).sum()

# Print the zero counts
print("Zero counts in junctions:")
print(junctions_zero_counts)

print("Zero counts in segments:")
print(segments_zero_counts)

In [ ]:
# Get county info
arcpy.analysis.SpatialJoin("segments_7", "counties_suncloud", r"safety_layers.gdb\segments_8", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,segments_7,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,segments_7,TARGET_FID,-1,-1;Join_Count_1 "Join_Count" true true false 4 Long 0 0,First,#,segments_7,Join_Count_1,-1,-1;TARGET_FID_1 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_7,TARGET_FID_1,-1,-1;Join_Count_12 "Join_Count" true true false 4 Long 0 0,First,#,segments_7,Join_Count_12,-1,-1;TARGET_FID_12 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_7,TARGET_FID_12,-1,-1;Join_Count_12_13 "Join_Count" true true false 4 Long 0 0,First,#,segments_7,Join_Count_12_13,-1,-1;TARGET_FID_12_13 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_7,TARGET_FID_12_13,-1,-1;Join_Count_12_13_14 "Join_Count" true true false 4 Long 0 0,First,#,segments_7,Join_Count_12_13_14,-1,-1;TARGET_FID_12_13_14 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_7,TARGET_FID_12_13_14,-1,-1;Join_Count_12_13_14_15 "Join_Count" true true false 4 Long 0 0,First,#,segments_7,Join_Count_12_13_14_15,-1,-1;TARGET_FID_12_13_14_15 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_7,TARGET_FID_12_13_14_15,-1,-1;Join_Count_12_13_14_15_16 "Join_Count" true true false 4 Long 0 0,First,#,segments_7,Join_Count_12_13_14_15_16,-1,-1;TARGET_FID_12_13_14_15_16 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_7,TARGET_FID_12_13_14_15_16,-1,-1;route_id "Route ID" true true false 32 Text 0 0,First,#,segments_7,route_id,0,32;functional_classification_code "Functional Classification Code" true true false 2 Short 0 0,First,#,segments_7,functional_classification_code,-1,-1;functional_classification "Functional Classification" true true false 255 Text 0 0,First,#,segments_7,functional_classification,0,255;crashes "crashes" true true false 8 Double 0 0,First,#,segments_7,crashes,-1,-1;injury_crashes "injury_crashes" true true false 8 Double 0 0,First,#,segments_7,injury_crashes,-1,-1;fatal_crashes "fatal_crashes" true true false 8 Double 0 0,First,#,segments_7,fatal_crashes,-1,-1;fatal_injury_crashes "fatal_injury_crashes" true true false 8 Double 0 0,First,#,segments_7,fatal_injury_crashes,-1,-1;non_motorized_fatal_injury_crashes "non_motorized_fatal_injury_crashes" true true false 8 Double 0 0,First,#,segments_7,non_motorized_fatal_injury_crashes,-1,-1;impaired_driving_crashes "impaired_driving_crashes" true true false 8 Double 0 0,First,#,segments_7,impaired_driving_crashes,-1,-1;Length_Miles "Length_Miles" true true false 8 Double 0 0,First,#,segments_7,Length_Miles,-1,-1;FromDate "FromDate" true true false 8 Date 0 0,First,#,segments_7,FromDate,-1,-1;ToDate "ToDate" true true false 8 Date 0 0,First,#,segments_7,ToDate,-1,-1;RouteId "RouteId" true true false 32 Text 0 0,First,#,segments_7,RouteId,0,32;FromMeasure "FromMeasure" true true false 8 Double 0 0,First,#,segments_7,FromMeasure,-1,-1;ToMeasure "ToMeasure" true true false 8 Double 0 0,First,#,segments_7,ToMeasure,-1,-1;AADT "AADT" true true false 4 Long 0 0,First,#,segments_7,AADT,-1,-1;InServiceDate "InServiceDate" true true false 8 Date 0 0,First,#,segments_7,InServiceDate,-1,-1;Shape_Length_1 "Shape_Length" true true false 8 Double 0 0,First,#,segments_7,Shape_Length_1,-1,-1;VMT "VMT" true true false 8 Double 0 0,First,#,segments_7,VMT,-1,-1;MVM "MVM" true true false 8 Double 0 0,First,#,segments_7,MVM,-1,-1;injury_rate "injury_rate" true true false 8 Double 0 0,First,#,segments_7,injury_rate,-1,-1;fatal_injuy_rate "fatal_injuy_rate" true true false 8 Double 0 0,First,#,segments_7,fatal_injuy_rate,-1,-1;UA_ID "URBAN AREA ID" true true false 5 Text 0 0,First,#,segments_7,UA_ID,0,5;NAME "NAME" true true false 60 Text 0 0,First,#,segments_7,NAME,0,60;LSAD "LEGAL/STATISTICAL AREA CODE (LSAD)" true true false 2 Text 0 0,First,#,segments_7,LSAD,0,2;LSAD_DESC "LSAD DESCRIPTION" true true false 15 Text 0 0,First,#,segments_7,LSAD_DESC,0,15;POP2010 "POP2010" true true false 4 Long 0 0,First,#,segments_7,POP2010,-1,-1;POP10_SQMI "POP10_SQMI" true true false 8 Double 0 0,First,#,segments_7,POP10_SQMI,-1,-1;HSE_UNITS "HOUSING UNITS" true true false 4 Long 0 0,First,#,segments_7,HSE_UNITS,-1,-1;SQMI "SQMI" true true false 8 Double 0 0,First,#,segments_7,SQMI,-1,-1;Shape_Length_12 "Shape_Length" true true false 8 Double 0 0,First,#,segments_7,Shape_Length_12,-1,-1;in_urban_area "in_urban_area" true true false 8 Double 0 0,First,#,segments_7,in_urban_area,-1,-1;RouteIsDivided "Route Is Divided" true true false 2 Short 0 0,First,#,segments_7,RouteIsDivided,-1,-1;FromDate_1 "FromDate" true true false 200 Text 0 0,First,#,segments_7,FromDate_1,0,200;ToDate_1 "ToDate" true true false 200 Text 0 0,First,#,segments_7,ToDate_1,0,200;RouteId_1 "RouteId" true true false 200 Text 0 0,First,#,segments_7,RouteId_1,0,200;FromMeasure_1 "FromMeasure" true true false 200 Text 0 0,First,#,segments_7,FromMeasure_1,0,200;ToMeasure_1 "ToMeasure" true true false 200 Text 0 0,First,#,segments_7,ToMeasure_1,0,200;MedianWidthFt "MedianWidthFt" true true false 200 Text 0 0,First,#,segments_7,MedianWidthFt,0,200;MedianType "MedianType" true true false 200 Text 0 0,First,#,segments_7,MedianType,0,200;NumberOfLanes "NumberOfLanes" true true false 200 Text 0 0,First,#,segments_7,NumberOfLanes,0,200;BeginWidthFt "BeginWidthFt" true true false 200 Text 0 0,First,#,segments_7,BeginWidthFt,0,200;EndWidthFt "EndWidthFt" true true false 200 Text 0 0,First,#,segments_7,EndWidthFt,0,200;OTTYear "OTTYear" true true false 200 Text 0 0,First,#,segments_7,OTTYear,0,200;AuxLaneType "AuxLaneType" true true false 200 Text 0 0,First,#,segments_7,AuxLaneType,0,200;AuxLaneTypeForSymbol "AuxLaneTypeForSymbol" true true false 200 Text 0 0,First,#,segments_7,AuxLaneTypeForSymbol,0,200;InServiceDate_1 "InServiceDate" true true false 200 Text 0 0,First,#,segments_7,InServiceDate_1,0,200;OutServiceDate "OutServiceDate" true true false 200 Text 0 0,First,#,segments_7,OutServiceDate,0,200;SourceYear "SourceYear" true true false 200 Text 0 0,First,#,segments_7,SourceYear,0,200;Shape_Length_12_13 "Shape_Length" true true false 8 Double 0 0,First,#,segments_7,Shape_Length_12_13,-1,-1;divided "divided" true true false 8 Double 0 0,First,#,segments_7,divided,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,segments_7,Shape_Length,-1,-1;num_lanes "num_lanes" true true false 8 Double 0 0,First,#,segments_7,num_lanes,-1,-1;Area_Acres "Area_Acres" true true false 8 Double 0 0,First,#,counties_suncloud,Area_Acres,-1,-1;NAME_1 "NAME" true true false 10 Text 0 0,First,#,counties_suncloud,NAME,0,10;CODE "CODE" true true false 2 Short 0 0,First,#,counties_suncloud,CODE,-1,-1;Peri_Mile "Peri_Mile" true true false 8 Double 0 0,First,#,counties_suncloud,Peri_Mile,-1,-1;Area_Miles "Area_Miles" true true false 8 Double 0 0,First,#,counties_suncloud,Area_Miles,-1,-1;Per_Mile "Per_Mile" true true false 8 Double 0 0,First,#,counties_suncloud,Per_Mile,-1,-1;Shape_Leng "Shape_Leng" true true false 8 Double 0 0,First,#,counties_suncloud,Shape_Leng,-1,-1;Shape_Length_12_13_14 "Shape_Length" false true true 8 Double 0 0,First,#,counties_suncloud,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,counties_suncloud,Shape_Area,-1,-1', "WITHIN", None, '')
arcpy.analysis.SpatialJoin("junctions_4", "counties_suncloud", r"safety_layers.gdb\junctions_5", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,junctions_4,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,junctions_4,TARGET_FID,-1,-1;Join_Count_1 "Join_Count" true true false 4 Long 0 0,First,#,junctions_4,Join_Count_1,-1,-1;TARGET_FID_1 "TARGET_FID" true true false 4 Long 0 0,First,#,junctions_4,TARGET_FID_1,-1,-1;Join_Count_12 "Join_Count" true true false 4 Long 0 0,First,#,junctions_4,Join_Count_12,-1,-1;TARGET_FID_12 "TARGET_FID" true true false 4 Long 0 0,First,#,junctions_4,TARGET_FID_12,-1,-1;Join_Count_12_13 "Join_Count" true true false 4 Long 0 0,First,#,junctions_4,Join_Count_12_13,-1,-1;TARGET_FID_12_13 "TARGET_FID" true true false 4 Long 0 0,First,#,junctions_4,TARGET_FID_12_13,-1,-1;FID_Junctions_SunCloud_Select "FID_Junctions_SunCloud_Select" true true false 4 Long 0 0,First,#,junctions_4,FID_Junctions_SunCloud_Select,-1,-1;FromDate "FromDate" true true false 8 Date 0 0,First,#,junctions_4,FromDate,-1,-1;ToDate "ToDate" true true false 8 Date 0 0,First,#,junctions_4,ToDate,-1,-1;RouteID "RouteID" true true false 32 Text 0 0,First,#,junctions_4,RouteID,0,32;Measure "Measure" true true false 8 Double 0 0,First,#,junctions_4,Measure,-1,-1;MasterJunc "MasterJunc" true true false 38 Text 0 0,First,#,junctions_4,MasterJunc,0,38;NumberOfLe "NumberOfLe" true true false 4 Long 0 0,First,#,junctions_4,NumberOfLe,-1,-1;JunctionGe "JunctionGe" true true false 4 Long 0 0,First,#,junctions_4,JunctionGe,-1,-1;JunctionsS "JunctionsS" true true false 2 Text 0 0,First,#,junctions_4,JunctionsS,0,2;TrafficCon "TrafficCon" true true false 4 Long 0 0,First,#,junctions_4,TrafficCon,-1,-1;TurnCode "TurnCode" true true false 10 Text 0 0,First,#,junctions_4,TurnCode,0,10;Detection "Detection" true true false 4 Long 0 0,First,#,junctions_4,Detection,-1,-1;Signalizat "Signalizat" true true false 4 Long 0 0,First,#,junctions_4,Signalizat,-1,-1;RailCrossi "RailCrossi" true true false 7 Text 0 0,First,#,junctions_4,RailCrossi,0,7;SchoolZone "SchoolZone" true true false 4 Long 0 0,First,#,junctions_4,SchoolZone,-1,-1;BicycleFac "BicycleFac" true true false 4 Long 0 0,First,#,junctions_4,BicycleFac,-1,-1;Crosswalk "Crosswalk" true true false 4 Long 0 0,First,#,junctions_4,Crosswalk,-1,-1;PctGreenTi "PctGreenTi" true true false 4 Long 0 0,First,#,junctions_4,PctGreenTi,-1,-1;PctGreen_1 "PctGreen_1" true true false 4 Long 0 0,First,#,junctions_4,PctGreen_1,-1,-1;SourceID "SourceID" true true false 4 Long 0 0,First,#,junctions_4,SourceID,-1,-1;JunctionMa "JunctionMa" true true false 4 Long 0 0,First,#,junctions_4,JunctionMa,-1,-1;JunctionTy "JunctionTy" true true false 4 Long 0 0,First,#,junctions_4,JunctionTy,-1,-1;OffsetInte "OffsetInte" true true false 1 Text 0 0,First,#,junctions_4,OffsetInte,0,1;OffsetIn_1 "OffsetIn_1" true true false 8 Double 0 0,First,#,junctions_4,OffsetIn_1,-1,-1;InServiceD "InServiceD" true true false 8 Date 0 0,First,#,junctions_4,InServiceD,-1,-1;FID_sun_cloud_routes "FID_sun_cloud_routes" true true false 4 Long 0 0,First,#,junctions_4,FID_sun_cloud_routes,-1,-1;route_id "route_id" true true false 32 Text 0 0,First,#,junctions_4,route_id,0,32;from_measu "from_measu" true true false 8 Double 0 0,First,#,junctions_4,from_measu,-1,-1;to_measure "to_measure" true true false 8 Double 0 0,First,#,junctions_4,to_measure,-1,-1;functional "functional" true true false 4 Long 0 0,First,#,junctions_4,functional,-1,-1;facility_t "facility_t" true true false 4 Long 0 0,First,#,junctions_4,facility_t,-1,-1;crashes "crashes" true true false 255 Text 0 0,First,#,junctions_4,crashes,0,255;injury_crashes "injury_crashes" true true false 8 Double 0 0,First,#,junctions_4,injury_crashes,-1,-1;fatal_crashes "fatal_crashes" true true false 8 Double 0 0,First,#,junctions_4,fatal_crashes,-1,-1;fatal_injury_crashes "fatal_injury_crashes" true true false 8 Double 0 0,First,#,junctions_4,fatal_injury_crashes,-1,-1;non_motorized_fatal_injury "non_motorized_fatal_injury" true true false 8 Double 0 0,First,#,junctions_4,non_motorized_fatal_injury,-1,-1;impaired_driving_crashes "impaired_driving_crashes" true true false 8 Double 0 0,First,#,junctions_4,impaired_driving_crashes,-1,-1;AADT "AADT" true true false 8 Double 0 0,First,#,junctions_4,AADT,-1,-1;aadt_major "aadt_major" true true false 8 Double 0 0,First,#,junctions_4,aadt_major,-1,-1;aadt_minor "aadt_minor" true true false 8 Double 0 0,First,#,junctions_4,aadt_minor,-1,-1;TEV "TEV" true true false 8 Double 0 0,First,#,junctions_4,TEV,-1,-1;MEV "MEV" true true false 8 Double 0 0,First,#,junctions_4,MEV,-1,-1;injury_rate "injury_rate" true true false 8 Double 0 0,First,#,junctions_4,injury_rate,-1,-1;fatal_injury_rate "fatal_injury_rate" true true false 8 Double 0 0,First,#,junctions_4,fatal_injury_rate,-1,-1;lanes_entering "lanes_entering" true true false 8 Double 0 0,First,#,junctions_4,lanes_entering,-1,-1;Area_Acres "Area_Acres" true true false 8 Double 0 0,First,#,counties_suncloud,Area_Acres,-1,-1;NAME "NAME" true true false 10 Text 0 0,First,#,counties_suncloud,NAME,0,10;CODE "CODE" true true false 2 Short 0 0,First,#,counties_suncloud,CODE,-1,-1;Peri_Mile "Peri_Mile" true true false 8 Double 0 0,First,#,counties_suncloud,Peri_Mile,-1,-1;Area_Miles "Area_Miles" true true false 8 Double 0 0,First,#,counties_suncloud,Area_Miles,-1,-1;Per_Mile "Per_Mile" true true false 8 Double 0 0,First,#,counties_suncloud,Per_Mile,-1,-1;Shape_Leng "Shape_Leng" true true false 8 Double 0 0,First,#,counties_suncloud,Shape_Leng,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,counties_suncloud,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,counties_suncloud,Shape_Area,-1,-1', "WITHIN", None, '')


In [ ]:
# To double
arcpy.management.CalculateField("segments_8", "aadt", "!AADT!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("segments_8", "is_divided", "!divided!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")


In [ ]:
# Assign spf ID numbers per segment

# Read the CSV file into a DataFrame
csv_path = "data/spfs/spf_segments.csv"
df = pd.read_csv(os.path.join(base_path, csv_path))

# Path to your feature class 
feature_class_path = "segments_8"  # Replace with your actual path

# Check if a field exists in a feature class or table
def field_exists(table, field):
    return field in [f.name for f in arcpy.ListFields(table)]

# Create the new field 'spf_id' if it doesn't already exist
if not field_exists(feature_class_path, 'spf_id'):
    arcpy.AddField_management(feature_class_path, "spf_id", "TEXT")

# Loop through each feature in the feature class
with arcpy.da.UpdateCursor(feature_class_path, ['num_lanes', 'is_divided', 'in_urban_area', 'spf_id']) as cursor:
    for row in cursor:
        lanes = row[0]
        divided = row[1]
        urban = row[2]

        # Iterate through the conditions in the CSV
        for index, csv_row in df.iterrows():
            num_lanes_cond = csv_row['num_lanes_cond'].replace("num_lanes", str(lanes))
            is_divided_cond = csv_row['is_divided_cond'].replace("is_divided", str(divided))
            is_urban_cond = csv_row['is_urban_cond'].replace("in_urban_area", str(urban))
            
            if eval(num_lanes_cond) and eval(is_divided_cond) and eval(is_urban_cond):
                row[3] = csv_row['spf_id']
                break

        # Update the feature
        cursor.updateRow(row)

print("Successfully populated the spf_id attribute based on conditions.")



In [ ]:
# Calculate predicted crashes for segments (AUTOMATED)

# Read the CSV file into a DataFrame
csv_path = "data/spfs/spf_segments.csv"
df = pd.read_csv(os.path.join(base_path, csv_path))

# Path to your feature class
feature_class_path = "segments_8"  # Replace with your actual path

# Add new fields if they don't exist
fields_to_add = ['predicted_crashes_SPF', 'overdispersion_parameter']
for field in fields_to_add:
    if field not in [f.name for f in arcpy.ListFields(feature_class_path)]:
        arcpy.AddField_management(feature_class_path, field, "DOUBLE")

# Loop through each feature in the feature class
with arcpy.da.UpdateCursor(feature_class_path, ['spf_id', 'aadt', 'Length_Miles', 'predicted_crashes_SPF', 'overdispersion_parameter']) as cursor:
    for row in cursor:
        spf_id = str(row[0])  # Ensure spf_id is a string
        
        # Skip if spf_id is 'no_spf_defined'
        if spf_id == 'no_spf_defined':
            continue
        
        aadt = row[1]
        length = row[2]
        
        # Fetch the corresponding equation and overdispersion for the current spf_id
        filtered_df = df[df['spf_id'].astype(str) == spf_id]
        
        if not filtered_df.empty:
            spf_equation = filtered_df['spf'].iloc[0]
            overdispersion = filtered_df['overdispersion'].iloc[0]
            row[4] = overdispersion  # Populate overdispersion_parameter
            
            # Use the equation to calculate 'predicted_crashes_SPF'
            if aadt is not None and length is not None and aadt > 0 and length > 0:
                equation_to_eval = spf_equation.replace("aadt", str(aadt)).replace("length", str(length))
                row[3] = eval(equation_to_eval)
            else:
                print(f"Skipped row due to invalid values or None: spf_id={spf_id}, aadt={aadt}, length={length}")
                
            # Update the feature
            cursor.updateRow(row)
        else:
            print(f"No equation found for spf_id: {spf_id}")

print("Successfully calculated and updated attributes.")



In [ ]:
# Overdisperson parameter to double
arcpy.management.CalculateField("segments_8", 
                                "overdispersionparameter", 
                                "!overdispersion_parameter!", 
                                "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

arcpy.management.CalculateField("segments_8", 
                                "predicted_crashes_SPF_value", 
                                "!predicted_crashes_SPF!", 
                                "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

In [ ]:
# To double
arcpy.management.CalculateField("junctions_5", "traffic_con", "!TrafficCon!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("junctions_5", "num_legs", "!NumberOfLe!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")


In [ ]:
# Identify SPFs for Junctions

# Input feature class and CSV path
feature_class_path = "junctions_5"
csv_path = "data/spfs/spf_junctions.csv"

# Load the SPF data from CSV into a DataFrame
df = pd.read_csv(os.path.join(base_path, csv_path))

# Create new field for SPF ID if it doesn't exist
fields = [f.name for f in arcpy.ListFields(feature_class_path)]
if 'spf_id' not in fields:
    arcpy.AddField_management(feature_class_path, 'spf_id', 'TEXT', field_length=10)

# Define the field names
fields = ['traffic_con', 'num_legs', 'spf_id']

# Update the feature class with the SPF IDs based on conditions
with arcpy.da.UpdateCursor(feature_class, fields) as cursor:
    for row in cursor:
        traffic_con = row[0]
        num_legs = row[1]
        spf_id = None

        # Iterate over each SPF condition in the DataFrame
        for index, row_df in df.iterrows():
            traffic_con_cond_expr = row_df['traffic_con_cond']
            num_legs_cond_expr = row_df['num_legs_cond']

            # Check if either expression is a float (meaning it's nan) or contains 'nan' and skip evaluation if so
            if isinstance(traffic_con_cond_expr, float) or isinstance(num_legs_cond_expr, float) or 'nan' in str(traffic_con_cond_expr) or 'nan' in str(num_legs_cond_expr):
                print(f"Skipping evaluation due to 'nan' or missing value: {traffic_con_cond_expr} or {num_legs_cond_expr}")
                continue

            try:
                traffic_con_cond = eval(traffic_con_cond_expr)
                num_legs_cond = eval(num_legs_cond_expr)
            except SyntaxError:
                print(f"Error evaluating: {traffic_con_cond_expr} or {num_legs_cond_expr}")
                continue

            # Check if conditions match, and if so, set the spf_id
            if traffic_con_cond and num_legs_cond:
                spf_id = row_df['spf_id']
                break


        # Update the SPF ID value in the feature class
        if spf_id is not None:
            row[-1] = spf_id
            cursor.updateRow(row)

print("Finished identifying SPF IDs for junctions!")



In [ ]:
# Change aadt_major and aadt_minor names to account for eval calculation

# Path to your feature class
feature_class = "junctions_5"

# Rename 'aadt_major' to 'major_aadt'
arcpy.AlterField_management(feature_class, 'aadt_major', 'major_aadt', 'major_aadt')

# Rename 'aadt_minor' to 'minor_aadt'
arcpy.AlterField_management(feature_class, 'aadt_minor', 'minor_aadt', 'minor_aadt')


In [ ]:
# Calculate predicted crashes for junctions (AUTOMATED)

# Read the CSV file into a DataFrame
csv_path = "data/spfs/spf_junctions.csv"
df = pd.read_csv(os.path.join(base_path, csv_path))

# Path to your feature class
feature_class_path = "junctions_5"  # Replace with your actual path

# Add new fields if they don't exist
fields_to_add = ['predicted_crashes_SPF', 'overdispersion_parameter']
for field in fields_to_add:
    if field not in [f.name for f in arcpy.ListFields(feature_class_path)]:
        arcpy.AddField_management(feature_class_path, field, "DOUBLE")
        
no_equation = 0
updated = 0
invalid_values = 0

# Loop through each feature in the feature class
with arcpy.da.UpdateCursor(feature_class_path, ['spf_id', 'aadt', 'major_aadt', 'minor_aadt', 'lanes_entering', 'predicted_crashes_SPF', 'overdispersion_parameter']) as cursor:
    for row in cursor:
        spf_id = str(row[0])  # Ensure spf_id is a string
        
        # Skip if spf_id is 'no_spf_defined'
        if spf_id == 'no_spf_defined':
            continue
        
        # Get values for aadt, aadt_major, aadt_minor, and lanes_entering
        aadt = row[1]
        major_aadt = row[2]
        minor_aadt = row[3]
        lanes_entering = row[4]

        # Fetch the corresponding equation and overdispersion for the current spf_id
        filtered_df = df[df['spf_id'].astype(str) == spf_id]
        
        if not filtered_df.empty:
            spf_equation = filtered_df['spf'].iloc[0]
            overdispersion = filtered_df['overdispersion'].iloc[0]
            row[6] = overdispersion  # Populate overdispersion_parameter
            
            # Use the equation to calculate 'predicted_crashes_SPF'
            if all([value is not None and value > 0 for value in [aadt, major_aadt, minor_aadt, lanes_entering]]):
                equation_to_eval = spf_equation.replace("major_aadt", str(major_aadt)).replace("minor_aadt", str(minor_aadt)).replace("aadt", str(aadt)).replace("lanes_entering", str(lanes_entering))

                try:
                    row[5] = eval(equation_to_eval)
                except SyntaxError:
                    print(f"Error evaluating equation for spf_id={spf_id}: {equation_to_eval}")
                    continue
                
            else:
                invalid_values = invalid_values + 1
                print(f"Skipped row due to invalid values or None: spf_id={spf_id}, aadt={aadt}, major_aadt={major_aadt}, minor_aadt={minor_aadt}, lanes_entering={lanes_entering}")
            
            # Update the feature
            updated = updated + 1
            cursor.updateRow(row)
        else:
            no_equation = no_equation + 1
            #print(f"No equation found for spf_id: {spf_id}")
print("")
print("Completed")
print(f"No equation found for {no_equation} junctions")
print(f"Skipped calculation due to invalid values for {invalid_values} junctions")
print(f"Successfully calculated {updated} junctions")


#### Predicted Crashes Calibration

In [ ]:
# In these steps we are calculating a common SPF for segments and junctions to find the relationship of KABC vs KABCO.

# Calculate the PAG SPF for calibration

# Segment Relationship Calibration
# Comparing PAG SPF vs ADOT SPF
arcpy.management.CalculateField("segments_8", 
                                "predicted_crashes_PAG", 
                                "math.exp(-4.45) * !aadt!**0.56 * !length_miles!", # PAG SPF for Rural Multi-Lane Divided (sc03)
                                "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

# Change back aadt_major and aadt_minor names
feature_class = "junctions_5"
arcpy.AlterField_management(feature_class, 'major_aadt', 'aadt_major', 'aadt_major')
arcpy.AlterField_management(feature_class, 'minor_aadt', 'aadt_minor', 'aadt_minor')

# Junction Relationship Calibration
# Comparing PAG SPF vs HSM Predefined
arcpy.management.CalculateField("junctions_5", 
                                "predicted_crashes_predefined", 
                                "math.exp(-11.02) * !aadt_major!**1.02 * !aadt_minor!**0.24", # Predefined SPF for 4-leg signalized (sc10)
                                "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")


In [ ]:
# Find the difference between a KABCO and KABC SPF

# Segment Calibration using PAG SPFs
arcpy.management.CalculateField("segments_8", 
                                "diff", 
                                "None if !predicted_crashes_SPF_value! == 0 else !predicted_crashes_PAG! / !predicted_crashes_SPF_value!", 
                                "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

# Junction Calibration using predefined
arcpy.management.CalculateField("junctions_5", 
                                "diff", 
                                "None if !predicted_crashes_SPF! == 0 else !predicted_crashes_predefined! / !predicted_crashes_SPF!", 
                                "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")


In [ ]:
# Query to kabco spfs and set a calibration factor of the mean diff for diff <= 1

# Load the CSV files into DataFrames
df_segments = pd.read_csv("data/spfs/spf_segments.csv")
df_junctions = pd.read_csv("data/spfs/spf_junctions.csv")

# Identify SPF IDs for KABCO type from the segments and junctions DataFrames
spf_ids_segments = df_segments[df_segments['type'] == 'kabco']['spf_id'].tolist()
spf_ids_junctions = df_junctions[df_junctions['type'] == 'kabco']['spf_id'].tolist()

# Initialize empty dictionaries to hold mean_values
mean_values_segments = {}
mean_values_junctions = {}

segments_layer = "segments_8"
junctions_layer = "junctions_5"

# Query and calculate mean for segments feature layer
for spf_id in spf_ids_segments:
    query_segments = f"spf_id = '{spf_id}'"
    arcpy.SelectLayerByAttribute_management(segments_layer, "NEW_SELECTION", query_segments)
    diff_values = [row[0] for row in arcpy.da.SearchCursor(segments_layer, "diff") if row[0] is not None and row[0] <= 1]
    mean_diff = sum(diff_values) / len(diff_values) if len(diff_values) > 0 else 1
    mean_values_segments[spf_id] = mean_diff

# Query and calculate mean for junctions feature layer
for spf_id in spf_ids_junctions:
    query_junctions = f"spf_id = '{spf_id}'"
    arcpy.SelectLayerByAttribute_management(junctions_layer, "NEW_SELECTION", query_junctions)
    diff_values = [row[0] for row in arcpy.da.SearchCursor(junctions_layer, "diff") if row[0] is not None and row[0] <= 1]
    mean_diff = sum(diff_values) / len(diff_values) if len(diff_values) > 0 else 1
    mean_values_junctions[spf_id] = mean_diff

# Create a list of all unique SPF IDs
all_spf_ids = list(set(df_segments['spf_id'].tolist() + df_junctions['spf_id'].tolist()))

# Initialize a dictionary to hold the calibration values for each SPF ID
calibration_values = {}

for spf_id in all_spf_ids:
    if spf_id in mean_values_segments.keys():
        calibration_values[spf_id] = mean_values_segments[spf_id]
    elif spf_id in mean_values_junctions.keys():
        calibration_values[spf_id] = mean_values_junctions[spf_id]
    else:
        calibration_values[spf_id] = 1

# Now you can update your CSV files with these calibration_values.
df_segments['kabc_calibration'] = df_segments['spf_id'].apply(lambda x: calibration_values.get(x, 1))
df_segments.to_csv('data/spfs/spf_segments.csv', index=False)

df_junctions['kabc_calibration'] = df_junctions['spf_id'].apply(lambda x: calibration_values.get(x, 1))
df_junctions.to_csv('data/spfs/spf_junctions.csv', index=False)


In [ ]:
# Assign averaged calibration to each segment or junction

# Clear selection from original calibration efforts
arcpy.management.SelectLayerByAttribute("segments_8", "CLEAR_SELECTION", '', None)
arcpy.management.SelectLayerByAttribute("junctions_5", "CLEAR_SELECTION", '', None)

# Convert calibration_values dictionary to a string representation
calibration_values_str = str(calibration_values)

# Field Calculation Function for segments_8 layer
arcpy.management.CalculateField("segments_8", "kabc_calibration", 
    "determine_value(!spf_id!, !NAME_1!)", "PYTHON3", f"""
calibration_values = {calibration_values_str}
def determine_value(spf_id, county):
    if county == 'PIMA':
        return 1
    else:
        return calibration_values.get(spf_id, 1)
""", "DOUBLE")

# Field Calculation Function for junctions_5 layer
arcpy.management.CalculateField("junctions_5", "kabc_calibration", 
    "determine_value(!spf_id!, !NAME!)", "PYTHON3", f"""
calibration_values = {calibration_values_str}
def determine_value(spf_id, county):
    if county == 'PIMA':
        return 1
    else:
        return calibration_values.get(spf_id, 1)
""", "DOUBLE")


In [ ]:
# Multiply calibration by original SPF value to get KABC crashes

# Predicted calibration calculation
arcpy.management.CalculateField("segments_8", "predicted_kabc_crashes", 
                                "!predicted_crashes_SPF_value!*!kabc_calibration!", 
                                "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

arcpy.management.CalculateField("junctions_5", "predicted_kabc_crashes", 
                                "!predicted_crashes_predefined!*!kabc_calibration!", 
                                "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

#### Expected, Excess, and LOSS Calculation

In [ ]:
# EB Weight
arcpy.management.CalculateField("segments_8", "EBweight_kabc", 
    "1 / (1 + (!predicted_kabc_crashes! * 5) / !overdispersionparameter!)", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

arcpy.management.CalculateField("junctions_5", "EBweight_kabc", 
    "1 / (1 + (!predicted_kabc_crashes! * 5) / !overdispersion_parameter!)", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

In [ ]:
# Expected Yearly Average Severe Crashes
arcpy.management.CalculateField("segments_8", "expected_kabc_crashes", 
    "!EBweight_kabc! * !predicted_kabc_crashes! + (1 - !EBweight_kabc!) * (!fatal_injury_crashes! / 5)", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

arcpy.management.CalculateField("junctions_5", "expected_kabc_crashes", 
    "!EBweight_kabc! * !predicted_kabc_crashes! + (1 - !EBweight_kabc!) * (!fatal_injury_crashes! / 5)", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

In [ ]:
# Excess Expected Severe Crashes
arcpy.management.CalculateField("segments_8", "excess_expected_kabc_crashes", 
    "!expected_kabc_crashes! - !predicted_kabc_crashes!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

arcpy.management.CalculateField("junctions_5", "excess_expected_kabc_crashes", 
    "!expected_kabc_crashes! - !predicted_kabc_crashes!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

In [ ]:
# LOSS Value
segments_expression = """
def calculate_loss(fatal_injury_crashes, expected_kabc_crashes):
    if expected_kabc_crashes != 0:
        return (fatal_injury_crashes / 5) / expected_kabc_crashes
    else:
        return 0
"""

junctions_expression = """
def calculate_loss(fatal_injury_crashes, expected_kabc_crashes):
    if expected_kabc_crashes != 0:
        return (fatal_injury_crashes / 5) / expected_kabc_crashes
    else:
        return 0
"""

arcpy.management.CalculateField("segments_8", "LOSS_value", "calculate_loss(!fatal_injury_crashes!, !expected_kabc_crashes!)", "PYTHON3", segments_expression, "DOUBLE", "NO_ENFORCE_DOMAINS")

arcpy.management.CalculateField("junctions_5", "LOSS_value", "calculate_loss(!fatal_injury_crashes!, !expected_kabc_crashes!)", "PYTHON3", junctions_expression, "DOUBLE", "NO_ENFORCE_DOMAINS")



In [ ]:
# Find the LOSS categories (part 1)
# Level of Service Safety (LOSS) is categorized in 4 levels. 
# LOSS I - Indicates low potential for crash reduction.
# LOSS II - Indicates low to moderate potential for crash reduction.
# LOSS III - Indicates moderate to high potential for crash reduction; and
# LOSS IV - Indicates high potential for crash reduction.

# In order to determine the categorization amount. 
# It's 1.5 times the standard deviation. 
# For example. If the standard deviation is 16, then:
# LOSS I is <-24
# LOSS II is -24 to <= 1
# LOSS III is >1 to <24
# LOSS IV is >= 24

# Finding standard deviation * 1.5

# Find the standard deviation function
def calculate_std_dev(table, field):
    values = [row[0] for row in arcpy.da.SearchCursor(table, [field]) if row[0] is not None]
    if len(values) == 0:
        return 0  # Return 0 or other default value if no valid records
    mean_value = sum(values) / len(values)
    variance = sum([((x - mean_value) ** 2) for x in values]) / len(values)
    std_dev = math.sqrt(variance)
    return std_dev


# Calculate standard deviation for 'LOSS_value' in segments and junctions
std_dev_segments = calculate_std_dev("segments_8", "LOSS_value")
std_dev_junctions = calculate_std_dev("junctions_5", "LOSS_value")

# Dynamic threshold
threshold_value_segments = 1.5 * std_dev_segments
threshold_value_junctions = 1.5 * std_dev_junctions

In [ ]:
# Find the LOSS categories (part 2)

# Assign categories based on dynamic threshold calculated in previous step

# For Segments
arcpy.management.CalculateField("segments_8", "level_of_safety_service_kabc", 
    "choose_spf(!LOSS_value!, {})".format(threshold_value_segments), "PYTHON3", """
def choose_spf(LOSS_value, threshold):
    if LOSS_value <= -threshold:
        return "LOSS I"
    elif LOSS_value > -threshold and LOSS_value <= 1:
        return "LOSS II"
    elif LOSS_value > 1 and LOSS_value < threshold:
        return "LOSS III"
    elif LOSS_value >= threshold:
        return "LOSS IV"
    else:
        return "Error"
""")

# For Junctions
arcpy.management.CalculateField("junctions_5", "level_of_safety_service_kabc", 
    "choose_spf(!LOSS_value!, {})".format(threshold_value_junctions), "PYTHON3", """
def choose_spf(LOSS_value, threshold):
    if LOSS_value <= -threshold:
        return "LOSS I"
    elif LOSS_value > -threshold and LOSS_value <= 1:
        return "LOSS II"
    elif LOSS_value > 1 and LOSS_value < threshold:
        return "LOSS III"
    elif LOSS_value >= threshold:
        return "LOSS IV"
    else:
        return "Error"
""")


### Clean up

In [ ]:
# Copy to new feature layer
arcpy.conversion.FeatureClassToFeatureClass("segments_8", r"safety_layers.gdb", "safety_segments", '', 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,segments_8,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,segments_8,TARGET_FID,-1,-1;Join_Count_1 "Join_Count" true true false 4 Long 0 0,First,#,segments_8,Join_Count_1,-1,-1;TARGET_FID_1 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_8,TARGET_FID_1,-1,-1;Join_Count_12 "Join_Count" true true false 4 Long 0 0,First,#,segments_8,Join_Count_12,-1,-1;TARGET_FID_12 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_8,TARGET_FID_12,-1,-1;Join_Count_12_13 "Join_Count" true true false 4 Long 0 0,First,#,segments_8,Join_Count_12_13,-1,-1;TARGET_FID_12_13 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_8,TARGET_FID_12_13,-1,-1;Join_Count_12_13_14 "Join_Count" true true false 4 Long 0 0,First,#,segments_8,Join_Count_12_13_14,-1,-1;TARGET_FID_12_13_14 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_8,TARGET_FID_12_13_14,-1,-1;Join_Count_12_13_14_15 "Join_Count" true true false 4 Long 0 0,First,#,segments_8,Join_Count_12_13_14_15,-1,-1;TARGET_FID_12_13_14_15 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_8,TARGET_FID_12_13_14_15,-1,-1;Join_Count_12_13_14_15_16 "Join_Count" true true false 4 Long 0 0,First,#,segments_8,Join_Count_12_13_14_15_16,-1,-1;TARGET_FID_12_13_14_15_16 "TARGET_FID" true true false 4 Long 0 0,First,#,segments_8,TARGET_FID_12_13_14_15_16,-1,-1;route_id "Route ID" true true false 32 Text 0 0,First,#,segments_8,route_id,0,32;functional_classification_code "Functional Classification Code" true true false 2 Short 0 0,First,#,segments_8,functional_classification_code,-1,-1;functional_classification "Functional Classification" true true false 255 Text 0 0,First,#,segments_8,functional_classification,0,255;crashes "crashes" true true false 8 Double 0 0,First,#,segments_8,crashes,-1,-1;injury_crashes "injury_crashes" true true false 8 Double 0 0,First,#,segments_8,injury_crashes,-1,-1;fatal_crashes "fatal_crashes" true true false 8 Double 0 0,First,#,segments_8,fatal_crashes,-1,-1;fatal_injury_crashes "fatal_injury_crashes" true true false 8 Double 0 0,First,#,segments_8,fatal_injury_crashes,-1,-1;non_motorized_fatal_injury_crashes "non_motorized_fatal_injury_crashes" true true false 8 Double 0 0,First,#,segments_8,non_motorized_fatal_injury_crashes,-1,-1;impaired_driving_crashes "impaired_driving_crashes" true true false 8 Double 0 0,First,#,segments_8,impaired_driving_crashes,-1,-1;Length_Miles "Length_Miles" true true false 8 Double 0 0,First,#,segments_8,Length_Miles,-1,-1;FromDate "FromDate" true true false 8 Date 0 0,First,#,segments_8,FromDate,-1,-1;ToDate "ToDate" true true false 8 Date 0 0,First,#,segments_8,ToDate,-1,-1;RouteId "RouteId" true true false 32 Text 0 0,First,#,segments_8,RouteId,0,32;FromMeasure "FromMeasure" true true false 8 Double 0 0,First,#,segments_8,FromMeasure,-1,-1;ToMeasure "ToMeasure" true true false 8 Double 0 0,First,#,segments_8,ToMeasure,-1,-1;AADT "AADT" true true false 4 Long 0 0,First,#,segments_8,AADT,-1,-1;InServiceDate "InServiceDate" true true false 8 Date 0 0,First,#,segments_8,InServiceDate,-1,-1;Shape_Length_1 "Shape_Length" true true false 8 Double 0 0,First,#,segments_8,Shape_Length_1,-1,-1;VMT "VMT" true true false 8 Double 0 0,First,#,segments_8,VMT,-1,-1;MVM "MVM" true true false 8 Double 0 0,First,#,segments_8,MVM,-1,-1;injury_rate "injury_rate" true true false 8 Double 0 0,First,#,segments_8,injury_rate,-1,-1;fatal_injuy_rate "fatal_injuy_rate" true true false 8 Double 0 0,First,#,segments_8,fatal_injuy_rate,-1,-1;UA_ID "URBAN AREA ID" true true false 5 Text 0 0,First,#,segments_8,UA_ID,0,5;NAME "NAME" true true false 60 Text 0 0,First,#,segments_8,NAME,0,60;LSAD "LEGAL/STATISTICAL AREA CODE (LSAD)" true true false 2 Text 0 0,First,#,segments_8,LSAD,0,2;LSAD_DESC "LSAD DESCRIPTION" true true false 15 Text 0 0,First,#,segments_8,LSAD_DESC,0,15;POP2010 "POP2010" true true false 4 Long 0 0,First,#,segments_8,POP2010,-1,-1;POP10_SQMI "POP10_SQMI" true true false 8 Double 0 0,First,#,segments_8,POP10_SQMI,-1,-1;HSE_UNITS "HOUSING UNITS" true true false 4 Long 0 0,First,#,segments_8,HSE_UNITS,-1,-1;SQMI "SQMI" true true false 8 Double 0 0,First,#,segments_8,SQMI,-1,-1;Shape_Length_12 "Shape_Length" true true false 8 Double 0 0,First,#,segments_8,Shape_Length_12,-1,-1;in_urban_area "in_urban_area" true true false 8 Double 0 0,First,#,segments_8,in_urban_area,-1,-1;RouteIsDivided "Route Is Divided" true true false 2 Short 0 0,First,#,segments_8,RouteIsDivided,-1,-1;FromDate_1 "FromDate" true true false 200 Text 0 0,First,#,segments_8,FromDate_1,0,200;ToDate_1 "ToDate" true true false 200 Text 0 0,First,#,segments_8,ToDate_1,0,200;RouteId_1 "RouteId" true true false 200 Text 0 0,First,#,segments_8,RouteId_1,0,200;FromMeasure_1 "FromMeasure" true true false 200 Text 0 0,First,#,segments_8,FromMeasure_1,0,200;ToMeasure_1 "ToMeasure" true true false 200 Text 0 0,First,#,segments_8,ToMeasure_1,0,200;MedianWidthFt "MedianWidthFt" true true false 200 Text 0 0,First,#,segments_8,MedianWidthFt,0,200;MedianType "MedianType" true true false 200 Text 0 0,First,#,segments_8,MedianType,0,200;NumberOfLanes "NumberOfLanes" true true false 200 Text 0 0,First,#,segments_8,NumberOfLanes,0,200;BeginWidthFt "BeginWidthFt" true true false 200 Text 0 0,First,#,segments_8,BeginWidthFt,0,200;EndWidthFt "EndWidthFt" true true false 200 Text 0 0,First,#,segments_8,EndWidthFt,0,200;OTTYear "OTTYear" true true false 200 Text 0 0,First,#,segments_8,OTTYear,0,200;AuxLaneType "AuxLaneType" true true false 200 Text 0 0,First,#,segments_8,AuxLaneType,0,200;AuxLaneTypeForSymbol "AuxLaneTypeForSymbol" true true false 200 Text 0 0,First,#,segments_8,AuxLaneTypeForSymbol,0,200;InServiceDate_1 "InServiceDate" true true false 200 Text 0 0,First,#,segments_8,InServiceDate_1,0,200;OutServiceDate "OutServiceDate" true true false 200 Text 0 0,First,#,segments_8,OutServiceDate,0,200;SourceYear "SourceYear" true true false 200 Text 0 0,First,#,segments_8,SourceYear,0,200;Shape_Length_12_13 "Shape_Length" true true false 8 Double 0 0,First,#,segments_8,Shape_Length_12_13,-1,-1;divided "divided" true true false 512 Text 0 0,First,#,segments_8,divided,0,512;num_lanes "num_lanes" true true false 8 Double 0 0,First,#,segments_8,num_lanes,-1,-1;predicted_crashes_SPF "predicted_crashes_SPF" true true false 512 Text 0 0,First,#,segments_8,predicted_crashes_SPF,0,512;COUNTYFP "COUNTYFP" true true false 3 Text 0 0,First,#,segments_8,COUNTYFP,0,3;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,segments_8,Shape_Length,-1,-1;is_divided "is_divided" true true false 8 Double 0 0,First,#,segments_8,is_divided,-1,-1;overdispersion_parameter "overdispersion_parameter" true true false 512 Text 0 0,First,#,segments_8,overdispersion_parameter,0,512;spf_id "spf_id" true true false 512 Text 0 0,First,#,segments_8,spf_id,0,512;overdispersionparameter "overdispersionparameter" true true false 8 Double 0 0,First,#,segments_8,overdispersionparameter,-1,-1;predicted_crashes_PAG "predicted_crashes_PAG" true true false 8 Double 0 0,First,#,segments_8,predicted_crashes_PAG,-1,-1;diff "diff" true true false 8 Double 0 0,First,#,segments_8,diff,-1,-1;predicted_crashed_SPF_value "predicted_crashed_SPF_value" true true false 8 Double 0 0,First,#,segments_8,predicted_crashed_SPF_value,-1,-1;predicted_crashes_SPF_value "predicted_crashes_SPF_value" true true false 8 Double 0 0,First,#,segments_8,predicted_crashes_SPF_value,-1,-1;severe_calibration "severe_calibration" true true false 8 Double 0 0,First,#,segments_8,severe_calibration,-1,-1;severe_cal "severe_cal" true true false 8 Double 0 0,First,#,segments_8,severe_cal,-1,-1;predicted_crashes "predicted_crashes" true true false 8 Double 0 0,First,#,segments_8,predicted_crashes,-1,-1;predicted_kabc_crashes "predicted_kabc_crashes" true true false 8 Double 0 0,First,#,segments_8,predicted_kabc_crashes,-1,-1;EBweight_kabc "EBweight_kabc" true true false 8 Double 0 0,First,#,segments_8,EBweight_kabc,-1,-1;expected_kabc_crashes "expected_kabc_crashes" true true false 8 Double 0 0,First,#,segments_8,expected_kabc_crashes,-1,-1;excess_expected_kabc_crashes "excess_expected_kabc_crashes" true true false 8 Double 0 0,First,#,segments_8,excess_expected_kabc_crashes,-1,-1;LOSS_value "LOSS_value" true true false 8 Double 0 0,First,#,segments_8,LOSS_value,-1,-1;level_of_safety_service_kabc "level_of_safety_service_kabc" true true false 512 Text 0 0,First,#,segments_8,level_of_safety_service_kabc,0,512', '')
arcpy.conversion.FeatureClassToFeatureClass("junctions_5", r"safety_layers.gdb", "safety_junctions", '', 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,junctions_5,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,junctions_5,TARGET_FID,-1,-1;Join_Count_1 "Join_Count" true true false 4 Long 0 0,First,#,junctions_5,Join_Count_1,-1,-1;TARGET_FID_1 "TARGET_FID" true true false 4 Long 0 0,First,#,junctions_5,TARGET_FID_1,-1,-1;Join_Count_12 "Join_Count" true true false 4 Long 0 0,First,#,junctions_5,Join_Count_12,-1,-1;TARGET_FID_12 "TARGET_FID" true true false 4 Long 0 0,First,#,junctions_5,TARGET_FID_12,-1,-1;Join_Count_12_13 "Join_Count" true true false 4 Long 0 0,First,#,junctions_5,Join_Count_12_13,-1,-1;TARGET_FID_12_13 "TARGET_FID" true true false 4 Long 0 0,First,#,junctions_5,TARGET_FID_12_13,-1,-1;Join_Count_12_13_14 "Join_Count" true true false 4 Long 0 0,First,#,junctions_5,Join_Count_12_13_14,-1,-1;TARGET_FID_12_13_14 "TARGET_FID" true true false 4 Long 0 0,First,#,junctions_5,TARGET_FID_12_13_14,-1,-1;FID_Junctions_SunCloud_Select "FID_Junctions_SunCloud_Select" true true false 4 Long 0 0,First,#,junctions_5,FID_Junctions_SunCloud_Select,-1,-1;FromDate "FromDate" true true false 8 Date 0 0,First,#,junctions_5,FromDate,-1,-1;ToDate "ToDate" true true false 8 Date 0 0,First,#,junctions_5,ToDate,-1,-1;RouteID "RouteID" true true false 32 Text 0 0,First,#,junctions_5,RouteID,0,32;Measure "Measure" true true false 8 Double 0 0,First,#,junctions_5,Measure,-1,-1;MasterJunc "MasterJunc" true true false 38 Text 0 0,First,#,junctions_5,MasterJunc,0,38;NumberOfLe "NumberOfLe" true true false 4 Long 0 0,First,#,junctions_5,NumberOfLe,-1,-1;JunctionGe "JunctionGe" true true false 4 Long 0 0,First,#,junctions_5,JunctionGe,-1,-1;JunctionsS "JunctionsS" true true false 2 Text 0 0,First,#,junctions_5,JunctionsS,0,2;TrafficCon "TrafficCon" true true false 4 Long 0 0,First,#,junctions_5,TrafficCon,-1,-1;TurnCode "TurnCode" true true false 10 Text 0 0,First,#,junctions_5,TurnCode,0,10;Detection "Detection" true true false 4 Long 0 0,First,#,junctions_5,Detection,-1,-1;Signalizat "Signalizat" true true false 4 Long 0 0,First,#,junctions_5,Signalizat,-1,-1;RailCrossi "RailCrossi" true true false 7 Text 0 0,First,#,junctions_5,RailCrossi,0,7;SchoolZone "SchoolZone" true true false 4 Long 0 0,First,#,junctions_5,SchoolZone,-1,-1;BicycleFac "BicycleFac" true true false 4 Long 0 0,First,#,junctions_5,BicycleFac,-1,-1;Crosswalk "Crosswalk" true true false 4 Long 0 0,First,#,junctions_5,Crosswalk,-1,-1;PctGreenTi "PctGreenTi" true true false 4 Long 0 0,First,#,junctions_5,PctGreenTi,-1,-1;PctGreen_1 "PctGreen_1" true true false 4 Long 0 0,First,#,junctions_5,PctGreen_1,-1,-1;SourceID "SourceID" true true false 4 Long 0 0,First,#,junctions_5,SourceID,-1,-1;JunctionMa "JunctionMa" true true false 4 Long 0 0,First,#,junctions_5,JunctionMa,-1,-1;JunctionTy "JunctionTy" true true false 4 Long 0 0,First,#,junctions_5,JunctionTy,-1,-1;OffsetInte "OffsetInte" true true false 1 Text 0 0,First,#,junctions_5,OffsetInte,0,1;OffsetIn_1 "OffsetIn_1" true true false 8 Double 0 0,First,#,junctions_5,OffsetIn_1,-1,-1;InServiceD "InServiceD" true true false 8 Date 0 0,First,#,junctions_5,InServiceD,-1,-1;FID_sun_cloud_routes "FID_sun_cloud_routes" true true false 4 Long 0 0,First,#,junctions_5,FID_sun_cloud_routes,-1,-1;route_id "route_id" true true false 32 Text 0 0,First,#,junctions_5,route_id,0,32;from_measu "from_measu" true true false 8 Double 0 0,First,#,junctions_5,from_measu,-1,-1;to_measure "to_measure" true true false 8 Double 0 0,First,#,junctions_5,to_measure,-1,-1;functional "functional" true true false 4 Long 0 0,First,#,junctions_5,functional,-1,-1;facility_t "facility_t" true true false 4 Long 0 0,First,#,junctions_5,facility_t,-1,-1;crashes "crashes" true true false 255 Text 0 0,First,#,junctions_5,crashes,0,255;injury_crashes "injury_crashes" true true false 8 Double 0 0,First,#,junctions_5,injury_crashes,-1,-1;fatal_crashes "fatal_crashes" true true false 8 Double 0 0,First,#,junctions_5,fatal_crashes,-1,-1;fatal_injury_crashes "fatal_injury_crashes" true true false 8 Double 0 0,First,#,junctions_5,fatal_injury_crashes,-1,-1;non_motorized_fatal_injury "non_motorized_fatal_injury" true true false 8 Double 0 0,First,#,junctions_5,non_motorized_fatal_injury,-1,-1;impaired_driving_crashes "impaired_driving_crashes" true true false 8 Double 0 0,First,#,junctions_5,impaired_driving_crashes,-1,-1;AADT "AADT" true true false 8 Double 0 0,First,#,junctions_5,AADT,-1,-1;aadt_major "aadt_major" true true false 8 Double 0 0,First,#,junctions_5,aadt_major,-1,-1;aadt_minor "aadt_minor" true true false 8 Double 0 0,First,#,junctions_5,aadt_minor,-1,-1;TEV "TEV" true true false 8 Double 0 0,First,#,junctions_5,TEV,-1,-1;MEV "MEV" true true false 8 Double 0 0,First,#,junctions_5,MEV,-1,-1;injury_rate "injury_rate" true true false 8 Double 0 0,First,#,junctions_5,injury_rate,-1,-1;fatal_injury_rate "fatal_injury_rate" true true false 8 Double 0 0,First,#,junctions_5,fatal_injury_rate,-1,-1;lanes_entering "lanes_entering" true true false 8 Double 0 0,First,#,junctions_5,lanes_entering,-1,-1;Area_Acres "Area_Acres" true true false 8 Double 0 0,First,#,junctions_5,Area_Acres,-1,-1;NAME "NAME" true true false 10 Text 0 0,First,#,junctions_5,NAME,0,10;CODE "CODE" true true false 2 Short 0 0,First,#,junctions_5,CODE,-1,-1;Peri_Mile "Peri_Mile" true true false 8 Double 0 0,First,#,junctions_5,Peri_Mile,-1,-1;Area_Miles "Area_Miles" true true false 8 Double 0 0,First,#,junctions_5,Area_Miles,-1,-1;Per_Mile "Per_Mile" true true false 8 Double 0 0,First,#,junctions_5,Per_Mile,-1,-1;Shape_Leng "Shape_Leng" true true false 8 Double 0 0,First,#,junctions_5,Shape_Leng,-1,-1;traffic_con "traffic_con" true true false 8 Double 0 0,First,#,junctions_5,traffic_con,-1,-1;num_legs "num_legs" true true false 8 Double 0 0,First,#,junctions_5,num_legs,-1,-1;predicted_crashes_SPF "predicted_crashes_SPF" true true false 8 Double 0 0,First,#,junctions_5,predicted_crashes_SPF,-1,-1;overdispersion_parameter "overdispersion_parameter" true true false 8 Double 0 0,First,#,junctions_5,overdispersion_parameter,-1,-1;spf_id "spf_id" true true false 512 Text 0 0,First,#,junctions_5,spf_id,0,512;predicted_crashes_predefined "predicted_crashes_predefined" true true false 8 Double 0 0,First,#,junctions_5,predicted_crashes_predefined,-1,-1;diff "diff" true true false 8 Double 0 0,First,#,junctions_5,diff,-1,-1;severe_calibration "severe_calibration" true true false 8 Double 0 0,First,#,junctions_5,severe_calibration,-1,-1;predicted_kabc_crashes "predicted_kabc_crashes" true true false 8 Double 0 0,First,#,junctions_5,predicted_kabc_crashes,-1,-1;EBweight_kabc "EBweight_kabc" true true false 8 Double 0 0,First,#,junctions_5,EBweight_kabc,-1,-1;expected_kabc_crashes "expected_kabc_crashes" true true false 8 Double 0 0,First,#,junctions_5,expected_kabc_crashes,-1,-1;excess_expected_kabc_crashes "excess_expected_kabc_crashes" true true false 8 Double 0 0,First,#,junctions_5,excess_expected_kabc_crashes,-1,-1;LOSS_value "LOSS_value" true true false 8 Double 0 0,First,#,junctions_5,LOSS_value,-1,-1;level_of_safety_service_kabc "level_of_safety_service_kabc" true true false 512 Text 0 0,First,#,junctions_5,level_of_safety_service_kabc,0,512', '')

# Delete duplicate junctions
arcpy.management.DeleteIdentical("safety_junctions", "Shape;MasterJunc", "150 Feet", 0)


In [ ]:
# Remove un-used attributes and rename

working_layer = 'safety_segments'

system_fields = [arcpy.Describe(working_layer).OIDFieldName, 'Shape', 'geom', 'Shape_Length']
current_fields = [f.name for f in arcpy.ListFields(working_layer) if not f.name in system_fields]

# Define tuples of CurrentFieldName, target_field_name, Alias
target_fields = [
    ("OBJECTID", "OBJECTID", "Object ID"),
    ("crashes", "crashes", "Total number of all crashes"),
    ("impaired_driving_crashes", "impaired_driving_crashes", "Number of impaired driving crashes"),
    ("injury_crashes", "injury_crashes", "Number of injuries"),
    ("injury_rate", "injury_rate", "Rate of injuries per million entering vehicles"),
    ("fatal_injury_crashes", "fatal_injury_crashes", "Number of fatalities"),
    ("fatal_injuy_rate", "fatal_injury_rate", "Rate of fatalities per million entering vehicles"),
    ("non_motorized_fatal_injury_crashes", "non_motorized_fatal_injury", "Number of non-motorized fatalities and serious injuries"),
    ("level_of_safety_service_kabc", "level_of_safety_service_kabc", "Level of Safety Service for only KABC crashes"),
    ("spf_id", "spf_id", "Safety Performance Function ID"),
    ("predicted_kabc_crashes", "predicted_kabc_crashes", "SPF-based Predicted Number of KABC Crashes"),
    ("expected_kabc_crashes", "expected_kabc_crashes", "Expected Number of KABC Crashes"),
    ("excess_expected_kabc_crashes", "excess_expected_kabc_crashes", "Excess Expected Number of KABC Crashes"),
    ("level_of_safety_service_kabc", "level_of_safety_service_kabc", "LOSS for KABC Crashes"),
    ("NAME", "county", "County")
]

# delete unused fields
delete_fields = [field for field in current_fields if not field in [a for a, b, c in target_fields]]
if delete_fields:
    arcpy.management.DeleteField(working_layer, delete_fields)
    
for current, new, alias in target_fields:
    # arcgis can't change capitalization ... a hacky workaround
    if new != current and new.lower() == current.lower():
        arcpy.management.AlterField(working_layer, current, current + "1")
        current = current + "1"
    arcpy.management.AlterField(working_layer, current, new, alias)

In [ ]:
# Remove un-used attributes and rename

working_layer = 'safety_junctions'

system_fields = [arcpy.Describe(working_layer).OIDFieldName, 'Shape', 'geom', 'Shape_Length']
current_fields = [f.name for f in arcpy.ListFields(working_layer) if not f.name in system_fields]

# Define tuples of CurrentFieldName, target_field_name, Alias
target_fields = [
    ("OBJECTID", "OBJECTID", "Object ID"),
    ("crashes", "crashes", "Total number of all crashes"),
    ("impaired_driving_crashes", "impaired_driving_crashes", "Number of impaired driving crashes"),
    ("injury_crashes", "injury_crashes", "Number of injuries"),
    ("injury_rate", "injury_rate", "Rate of injuries per million entering vehicles"),
    ("fatal_injury_crashes", "fatal_injury_crashes", "Number of fatalities"),
    ("fatal_injury_rate", "fatal_injury_rate", "Rate of fatalities per million entering vehicles"),
    ("non_motorized_fatal_injury", "non_motorized_fatal_injury", "Number of non-motorized fatalities and serious injuries"),
    ("level_of_safety_service_kabc", "level_of_safety_service_kabc", "Level of Safety Service for only KABC crashes"),
    ("spf_id", "spf_id", "Safety Performance Function ID"),
    ("predicted_kabc_crashes", "predicted_kabc_crashes", "SPF-based Predicted Number of KABC Crashes"),
    ("expected_kabc_crashes", "expected_kabc_crashes", "Expected Number of KABC Crashes"),
    ("excess_expected_kabc_crashes", "excess_expected_kabc_crashes", "Excess Expected Number of KABC Crashes"),
    ("level_of_safety_service_kabc", "level_of_safety_service_kabc", "LOSS for KABC Crashes"),
    ("NAME", "county", "County")
]

# delete unused fields
delete_fields = [field for field in current_fields if not field in [a for a, b, c in target_fields]]
if delete_fields:
    arcpy.management.DeleteField(working_layer, delete_fields)
    
for current, new, alias in target_fields:
    # arcgis can't change capitalization ... a hacky workaround
    if new != current and new.lower() == current.lower():
        arcpy.management.AlterField(working_layer, current, current + "1")
        current = current + "1"
    arcpy.management.AlterField(working_layer, current, new, alias)
    

In [ ]:
print("----- END OF SCRIPT -----")

END OF SCRIPT